# "저공해차 보급촉진을 위한 제도운영 지원 및 개선방안 마련" 데이터 분석

## library

In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

### data load

In [235]:
total_data = pd.DataFrame()
# 2020년
count = 0
for m in range(1, 13):
    y = 20
    s = f'raw/{y}년/20{y}년{m:0>2}월.xlsx'
    m_data = pd.read_excel(s)
    count += m_data.shape[0]
    total_data = pd.concat([total_data, m_data], ignore_index=True)
# 2021년
for m in range(1, 13):
    y = 21
    s = f'raw/{y}년/{y}년{m:0>2}월.xlsx'
    m_data = pd.read_excel(s)
    count += m_data.shape[0]
    total_data = pd.concat([total_data, m_data], ignore_index=True)
# 2022년
for m in range(1, 9):
    y = 22
    s = f'raw/{y}년/{y}년{m:0>2}월.xlsx'
    m_data = pd.read_excel(s)
    count += m_data.shape[0]
    total_data = pd.concat([total_data, m_data], ignore_index=True)

In [236]:
print(f'개별 데이터 개수 합산 : {count}')
print(f'총 데이터 개수 : {total_data.shape[0]}')

개별 데이터 개수 합산 : 11665822
총 데이터 개수 : 11665822


In [78]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 12 columns):
 #   Column     Dtype  
---  ------     -----  
 0   충전소명       object 
 1   충전기ID      int64  
 2   지역         object 
 3   시군구        object 
 4   주소         object 
 5   충전기용량(kW)  float64
 6   충전기타입      object 
 7   충전시작일시     int64  
 8   충전종료일시     int64  
 9   충전시간       object 
 10  충전량        float64
 11  충전용량       object 
dtypes: float64(2), int64(3), object(7)
memory usage: 1.0+ GB


##### 20-22년 병합 데이터 출력

In [80]:
total_data.to_csv('2020-22년_급속충전.csv', encoding='utf-8')

##### 20-22년 병합 데이터 로드(다시 시작)

In [638]:
# total_data = pd.read_csv('2020-22년_급속충전.csv', index_col=0)
# total_data.info()

C:\Users\kbj\AppData\Local\Temp\ipykernel_17248\1573190630.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  total_data = pd.read_csv('2020-22년_급속충전.csv')


In [639]:
df = total_data.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 13 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   충전소명        object 
 2   충전기ID       int64  
 3   지역          object 
 4   시군구         object 
 5   주소          object 
 6   충전기용량(kW)   float64
 7   충전기타입       object 
 8   충전시작일시      int64  
 9   충전종료일시      int64  
 10  충전시간        object 
 11  충전량         float64
 12  충전용량        object 
dtypes: float64(2), int64(4), object(7)
memory usage: 1.1+ GB


In [640]:
df = df.drop(['Unnamed: 0'], axis=1)

## 전처리 & 분석
- 충전소명 : O
- 충전기ID : int64
- 지역 : O
- 시군구 : O
- 주소 : O
- 충전기용량(kW) : float64
- 충전기타입 : object 
- 충전시작일시 : int64  
- 충전종료일시 : int64  
- 충전시간 : object 
- 충전량 : float64
- 충전용량 : object 

#### 특이사항
- 2020년 12월까지 충전기용량(kW)가 2021년 1월부터 충전용량으로 변경
    - 2020년 12월까지 데이터의 충전기용량(kW) 값을 충전용량으로 이동

In [641]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 12 columns):
 #   Column     Dtype  
---  ------     -----  
 0   충전소명       object 
 1   충전기ID      int64  
 2   지역         object 
 3   시군구        object 
 4   주소         object 
 5   충전기용량(kW)  float64
 6   충전기타입      object 
 7   충전시작일시     int64  
 8   충전종료일시     int64  
 9   충전시간       object 
 10  충전량        float64
 11  충전용량       object 
dtypes: float64(2), int64(3), object(7)
memory usage: 1.0+ GB


In [642]:
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,NaN
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,NaN
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,NaN
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,NaN
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,NaN


In [645]:
df[df['충전용량'].notnull() == True].head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량
2796614,군위휴게소(부산방향),1,경상북도,군위군,경상북도 군위군 군위읍 경북대로 4084 (오곡리),NaN,DC차데모+AC3상+DC콤보,20210101092250,20210101095816,00:29:24,11.95,50.0
2796615,충주휴게소(마산방향),1,충청북도,충주시,충청북도 충주시 중앙탑면 중부내륙고속도로 231 (용전리),NaN,DC차데모+AC3상+DC콤보,20210101095126,20210101095818,00:06:48,4.86,50.0
2796616,내서읍스포츠센터,1,경상남도,창원시,경상남도 창원시 마산회원구 내서읍 중리 391-3,NaN,DC콤보,20210101094752,20210101095819,00:10:27,3.29,100.0
2796617,삼성혈 인근 공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 이도1동 1293-1,NaN,DC차데모+AC3상+DC콤보,20210101091323,20210101095819,00:40:00,7.52,50.0
2796618,본오1동행정복지센터,1,경기도,안산시,경기도 안산시 상록구 샘골로 16,NaN,DC콤보,20210101091559,20210101095835,00:40:00,21.66,100.0


### 충전시작시간 정리

In [646]:
# 충전시작일시의 데이터는 년~초 까지 모두 존재
temp = df['충전시작일시'].to_list()
temp2 = [str(i) for i in temp]
count = 0
for one in temp2:
    if len(one) == 8:
        count += 1
print(count)

0


In [647]:
ch_start = df['충전시작일시'].copy()
df['충전시작_년도'] = ch_start.astype(str).str[:4].astype(int)
df['충전시작_월'] = df['충전시작일시'].astype(str).str[4:6].astype(int)
df['충전시작_일'] = df['충전시작일시'].astype(str).str[6:8].astype(int)
df['충전시작_시'] = df['충전시작일시'].astype(str).str[8:10].astype(int)
df['충전시작_분'] = df['충전시작일시'].astype(str).str[10:12].astype(int)
df['충전시작_초'] = df['충전시작일시'].astype(str).str[12:14].astype(int)
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,NaN,2020,1,17,19,33,31
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,NaN,2020,1,30,14,35,45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,NaN,2020,1,5,0,57,28
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,NaN,2020,1,20,12,2,15
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,NaN,2020,1,23,8,54,17


In [648]:
df['충전시작일시(변환)'] = df['충전시작_년도'].astype(str) + '-' +  df['충전시작_월'].astype(str) + '-' + df['충전시작_일'].astype(str) + ' ' + df['충전시작_시'].astype(str) + ':' + df['충전시작_분'].astype(str) + ':' + df['충전시작_초'].astype(str)
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,NaN,2020,1,17,19,33,31,2020-1-17 19:33:31
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,NaN,2020,1,30,14,35,45,2020-1-30 14:35:45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,NaN,2020,1,5,0,57,28,2020-1-5 0:57:28
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,NaN,2020,1,20,12,2,15,2020-1-20 12:2:15
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,NaN,2020,1,23,8,54,17,2020-1-23 8:54:17


In [649]:
df['충전시작일시(변환)'] = pd.to_datetime(df['충전시작일시(변환)'])
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,NaN,2020,1,17,19,33,31,2020-01-17 19:33:31
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,NaN,2020,1,30,14,35,45,2020-01-30 14:35:45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,NaN,2020,1,5,0,57,28,2020-01-05 00:57:28
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,NaN,2020,1,20,12,2,15,2020-01-20 12:02:15
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,NaN,2020,1,23,8,54,17,2020-01-23 08:54:17


### 충전종료시간 정리

In [650]:
df[df['충전종료일시'] == ' ']

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)


In [651]:
# 충전종료일시의 데이터는 년~일까지만 있는 데이터가 71311개 존재
    # 해당 데이터 뒤에 시분초를 '000000'으로 추가하는 작업 필요함.
temp = df['충전종료일시'].to_list()
temp2 = [str(i) for i in temp]
count = 0
for one in temp2:
    if len(one) == 8:
        count += 1
print(count)

71311


In [652]:
df.tail()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)
11665817,죽전휴게소(서울방향),1,경기도,용인시,경기도 용인시 수지구 경부고속도로 400 (죽전동),NaN,DC차데모+AC3상+DC콤보,20220831235918,20220901004034,00:40:00,31.09,급속(50kW),2022,8,31,23,59,18,2022-08-31 23:59:18
11665818,원주모범운전자회,21,강원도,원주시,강원도 원주시 단구로 150,NaN,DC콤보,20220831235920,20220901001904,00:19:43,46.51,급속(200kW동시),2022,8,31,23,59,20,2022-08-31 23:59:20
11665819,검단2공영주차장,1,인천광역시,서구,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,NaN,DC차데모+AC3상+DC콤보,20220831235931,20220901004251,00:40:00,21.97,급속(50kW),2022,8,31,23,59,31,2022-08-31 23:59:31
11665820,강당골 공영주차장,1,대구광역시,남구,대구광역시 남구 봉덕동 1272-18,NaN,DC차데모+AC3상+DC콤보,20220831235934,20220901004057,00:40:00,27.60,급속(50kW),2022,8,31,23,59,34,2022-08-31 23:59:34
11665821,마천동2공영주차장,1,서울특별시,송파구,서울특별시 송파구 성내천로299-7,NaN,DC차데모+AC3상+DC콤보,20220831235946,20220901003700,00:34:09,9.77,급속(100kW멀티),2022,8,31,23,59,46,2022-08-31 23:59:46


In [653]:
temp = df['충전종료일시'].to_list()

In [654]:
for i, one in enumerate(temp):
    if len(str(one)) == 8:
        print(i)
        break

1578


In [655]:
temp[1578]

20200118

In [656]:
temp2 = [str(i) for i in temp]
temp2[1578]

'20200118'

In [657]:
for i, one in enumerate(temp2):
    if len(one) == 8:
        temp = one + '000000'
        temp2[i] = temp

In [658]:
temp2[1578]

'20200118000000'

In [659]:
df['충전종료일시'] = temp2
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,NaN,2020,1,17,19,33,31,2020-01-17 19:33:31
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,NaN,2020,1,30,14,35,45,2020-01-30 14:35:45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,NaN,2020,1,5,0,57,28,2020-01-05 00:57:28
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,NaN,2020,1,20,12,2,15,2020-01-20 12:02:15
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,NaN,2020,1,23,8,54,17,2020-01-23 08:54:17


In [660]:
df.iloc[1578, :]

충전소명                동해웰빙레포츠타운 주차장
충전기ID                          21
지역                            강원도
시군구                           동해시
주소                 강원도 동해시 덕골길 10
충전기용량(kW)                   200.0
충전기타입                        DC콤보
충전시작일시             20200118162253
충전종료일시             20200118000000
충전시간                     00:06:49
충전량                          1.91
충전용량                          NaN
충전시작_년도                      2020
충전시작_월                          1
충전시작_일                         18
충전시작_시                         16
충전시작_분                         22
충전시작_초                         53
충전시작일시(변환)    2020-01-18 16:22:53
Name: 1578, dtype: object

In [661]:
ch_time = df['충전종료일시'].copy()
df['충전종료_년도'] = ch_time.astype(str).str[:4].astype(int)
df['충전종료_월'] = ch_time.astype(str).str[4:6].astype(int)
df['충전종료_일'] = ch_time.astype(str).str[6:8].astype(int)
df['충전종료_시'] = ch_time.astype(str).str[8:10].astype(int)
df['충전종료_분'] = ch_time.astype(str).str[10:12].astype(int)
df['충전종료_초'] = ch_time.astype(str).str[12:14].astype(int)
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,...,19,33,31,2020-01-17 19:33:31,2020,1,17,20,14,17
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,...,14,35,45,2020-01-30 14:35:45,2020,1,30,15,6,45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,...,0,57,28,2020-01-05 00:57:28,2020,1,5,1,13,29
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,...,12,2,15,2020-01-20 12:02:15,2020,1,20,12,47,8
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,...,8,54,17,2020-01-23 08:54:17,2020,1,23,9,25,54


In [662]:
df.tail()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초
11665817,죽전휴게소(서울방향),1,경기도,용인시,경기도 용인시 수지구 경부고속도로 400 (죽전동),NaN,DC차데모+AC3상+DC콤보,20220831235918,20220901004034,00:40:00,...,23,59,18,2022-08-31 23:59:18,2022,9,1,0,40,34
11665818,원주모범운전자회,21,강원도,원주시,강원도 원주시 단구로 150,NaN,DC콤보,20220831235920,20220901001904,00:19:43,...,23,59,20,2022-08-31 23:59:20,2022,9,1,0,19,4
11665819,검단2공영주차장,1,인천광역시,서구,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,NaN,DC차데모+AC3상+DC콤보,20220831235931,20220901004251,00:40:00,...,23,59,31,2022-08-31 23:59:31,2022,9,1,0,42,51
11665820,강당골 공영주차장,1,대구광역시,남구,대구광역시 남구 봉덕동 1272-18,NaN,DC차데모+AC3상+DC콤보,20220831235934,20220901004057,00:40:00,...,23,59,34,2022-08-31 23:59:34,2022,9,1,0,40,57
11665821,마천동2공영주차장,1,서울특별시,송파구,서울특별시 송파구 성내천로299-7,NaN,DC차데모+AC3상+DC콤보,20220831235946,20220901003700,00:34:09,...,23,59,46,2022-08-31 23:59:46,2022,9,1,0,37,0


#### 충전종료일시-초 오류 데이터
- 1495개

In [663]:
df[df['충전종료_초'] >= 60].shape

(1495, 25)

In [664]:
over_s_idx = df[df['충전종료_초'] >= 60].index
len(over_s_idx)

1495

In [665]:
for i in over_s_idx:
    df.loc[i, '충전종료_분'] += 1
    df.loc[i, '충전종료_초'] -= 60

In [666]:
df.iloc[1960560, :]

충전소명                  포천시의회 야외주차장
충전기ID                          44
지역                            경기도
시군구                           포천시
주소                 경기도 포천시 중앙로 87
충전기용량(kW)                   100.0
충전기타입                        DC콤보
충전시작일시             20201023102909
충전종료일시             20201023103099
충전시간                     00:03:10
충전량                          2.36
충전용량                          NaN
충전시작_년도                      2020
충전시작_월                         10
충전시작_일                         23
충전시작_시                         10
충전시작_분                         29
충전시작_초                          9
충전시작일시(변환)    2020-10-23 10:29:09
충전종료_년도                      2020
충전종료_월                         10
충전종료_일                         23
충전종료_시                         10
충전종료_분                         31
충전종료_초                         39
Name: 1960560, dtype: object

#### 충전종료일시-분 오류 데이터
- 3270개

In [667]:
df[df['충전종료_분'] >= 60].shape

(3270, 25)

In [668]:
over_m_idx = df[df['충전종료_분'] >= 60].index
len(over_m_idx)

3270

In [669]:
for i in over_m_idx:
    df.loc[i, '충전종료_시'] += 1
    df.loc[i, '충전종료_분'] -= 60

In [670]:
df.iloc[1964993, :]

충전소명                  포천시의회 야외주차장
충전기ID                          45
지역                            경기도
시군구                           포천시
주소                 경기도 포천시 중앙로 87
충전기용량(kW)                   100.0
충전기타입                        DC콤보
충전시작일시             20201023174301
충전종료일시             20201023176497
충전시간                     00:36:36
충전량                         25.44
충전용량                          NaN
충전시작_년도                      2020
충전시작_월                         10
충전시작_일                         23
충전시작_시                         17
충전시작_분                         43
충전시작_초                          1
충전시작일시(변환)    2020-10-23 17:43:01
충전종료_년도                      2020
충전종료_월                         10
충전종료_일                         23
충전종료_시                         18
충전종료_분                          5
충전종료_초                         37
Name: 1964993, dtype: object

#### 충전종료일시-시 오류 데이터
- 71개

In [671]:
df[df['충전종료_시'] >= 24].shape

(71, 25)

In [672]:
over_h_idx = df[df['충전종료_시'] >= 24].index
len(over_h_idx)

71

In [673]:
for i in over_h_idx:
    df.loc[i, '충전종료_일'] += 1
    df.loc[i, '충전종료_시'] -= 24

In [674]:
df.iloc[2168170, :]

충전소명             제천종합운동장 대형버스 주차장
충전기ID                          41
지역                           충청북도
시군구                           제천시
주소               충청북도 제천시 숭의로 101
충전기용량(kW)                   100.0
충전기타입                        DC콤보
충전시작일시             20201030234823
충전종료일시             20201030237223
충전시간                     00:40:00
충전량                         23.22
충전용량                          NaN
충전시작_년도                      2020
충전시작_월                         10
충전시작_일                         30
충전시작_시                         23
충전시작_분                         48
충전시작_초                         23
충전시작일시(변환)    2020-10-30 23:48:23
충전종료_년도                      2020
충전종료_월                         10
충전종료_일                         31
충전종료_시                          0
충전종료_분                         12
충전종료_초                         23
Name: 2168170, dtype: object

#### 충전종료일시-일 오류 데이터
- 3개(index) : 3232880, 2440150, 2775050

In [675]:
df[(df['충전종료_월'] == 1) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [676]:
df[(df['충전종료_년도'] == 2021) & (df['충전종료_월'] == 2) & (df['충전종료_일'] > 28)].index

Int64Index([3232880], dtype='int64')

In [677]:
df[(df['충전종료_년도'] == 2022) & (df['충전종료_월'] == 2) & (df['충전종료_일'] > 28)].index

Int64Index([], dtype='int64')

In [678]:
# 2020년은 윤년
df[(df['충전종료_년도'] == 2020) & (df['충전종료_월'] == 2) & (df['충전종료_일'] > 29)].index

Int64Index([], dtype='int64')

In [679]:
df[(df['충전종료_월'] == 3) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [680]:
df[(df['충전종료_월'] == 4) & (df['충전종료_일'] > 30)].index

Int64Index([], dtype='int64')

In [681]:
df[(df['충전종료_월'] == 5) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [682]:
df[(df['충전종료_월'] == 6) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [683]:
df[(df['충전종료_월'] == 7) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [684]:
df[(df['충전종료_월'] == 8) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [685]:
df[(df['충전종료_월'] == 9) & (df['충전종료_일'] > 30)].index

Int64Index([], dtype='int64')

In [686]:
df[(df['충전종료_월'] == 10) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [687]:
df[(df['충전종료_월'] == 11) & (df['충전종료_일'] > 30)].index

Int64Index([2440150], dtype='int64')

In [688]:
df[(df['충전종료_월'] == 12) & (df['충전종료_일'] > 31)].index

Int64Index([2775050], dtype='int64')

In [689]:
check_col = ['충전종료_년도', '충전종료_월', '충전종료_일']
df.loc[3232880, check_col]

충전종료_년도    2021
충전종료_월        2
충전종료_일       29
Name: 3232880, dtype: object

In [690]:
df.loc[3232880, '충전종료_월'] = 3
df.loc[3232880, '충전종료_일'] = 1
df.loc[3232880, check_col]

충전종료_년도    2021
충전종료_월        3
충전종료_일        1
Name: 3232880, dtype: object

In [691]:
df.loc[2440150, check_col]

충전종료_년도    2020
충전종료_월       11
충전종료_일       31
Name: 2440150, dtype: object

In [692]:
df.loc[2440150, '충전종료_월'] = 12
df.loc[2440150, '충전종료_일'] = 1
df.loc[2440150, check_col]

충전종료_년도    2020
충전종료_월       12
충전종료_일        1
Name: 2440150, dtype: object

In [693]:
df.loc[2775050, check_col]

충전종료_년도    2020
충전종료_월       12
충전종료_일       32
Name: 2775050, dtype: object

In [694]:
df.loc[2775050, '충전종료_년도'] = 2021
df.loc[2775050, '충전종료_월'] = 1
df.loc[2775050, '충전종료_일'] = 1
df.loc[2775050, check_col]

충전종료_년도    2021
충전종료_월        1
충전종료_일        1
Name: 2775050, dtype: object

#### 충전종료시간(변환)

In [695]:
df['충전종료일시(변환)'] = df['충전종료_년도'].astype(str) + '-' +  df['충전종료_월'].astype(str) + '-' + df['충전종료_일'].astype(str) + ' ' + df['충전종료_시'].astype(str) + ':' + df['충전종료_분'].astype(str) + ':' + df['충전종료_초'].astype(str)
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,...,33,31,2020-01-17 19:33:31,2020,1,17,20,14,17,2020-1-17 20:14:17
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,...,35,45,2020-01-30 14:35:45,2020,1,30,15,6,45,2020-1-30 15:6:45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,...,57,28,2020-01-05 00:57:28,2020,1,5,1,13,29,2020-1-5 1:13:29
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,...,2,15,2020-01-20 12:02:15,2020,1,20,12,47,8,2020-1-20 12:47:8
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,...,54,17,2020-01-23 08:54:17,2020,1,23,9,25,54,2020-1-23 9:25:54


In [696]:
df['충전종료일시(변환)'] = pd.to_datetime(df['충전종료일시(변환)'])

In [697]:
df['충전시간(계산)'] = df['충전종료일시(변환)'] - df['충전시작일시(변환)']
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,...,31,2020-01-17 19:33:31,2020,1,17,20,14,17,2020-01-17 20:14:17,0 days 00:40:46
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,...,45,2020-01-30 14:35:45,2020,1,30,15,6,45,2020-01-30 15:06:45,0 days 00:31:00
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,...,28,2020-01-05 00:57:28,2020,1,5,1,13,29,2020-01-05 01:13:29,0 days 00:16:01
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,...,15,2020-01-20 12:02:15,2020,1,20,12,47,8,2020-01-20 12:47:08,0 days 00:44:53
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,...,17,2020-01-23 08:54:17,2020,1,23,9,25,54,2020-01-23 09:25:54,0 days 00:31:37


### 충전기용량과 충전용량 합치기

In [698]:
df[df['충전시작_년도'] == 2020].tail()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
2796609,탄천제2호 공영주차장,1,서울특별시,강남구,"서울특별시 강남구 일원동 4-49 , 주차장 입구 오른쪽",50.0,DC차데모+AC3상+DC콤보,20201219120138,20201219124144,00:40:00,...,38,2020-12-19 12:01:38,2020,12,19,12,41,44,2020-12-19 12:41:44,0 days 00:40:06
2796610,온천장역 공영주차장,1,부산광역시,동래구,부산광역시 동래구 중앙대로 1495-10 공영주차장,50.0,DC차데모+AC3상+DC콤보,20201219120210,20201219124217,00:40:00,...,10,2020-12-19 12:02:10,2020,12,19,12,42,17,2020-12-19 12:42:17,0 days 00:40:07
2796611,화서휴게소(상주방향),1,경상북도,상주시,경상북도 상주시 화서면 터골길 51-25,50.0,DC차데모+AC3상+DC콤보,20201219120513,20201219124254,00:37:28,...,13,2020-12-19 12:05:13,2020,12,19,12,42,54,2020-12-19 12:42:54,0 days 00:37:41
2796612,삼양동주민센터,1,제주특별자치도,제주시,"제주특별자치도 제주시 지석13길 6 , 주차장",50.0,DC차데모+AC3상+DC콤보,20201219121419,20201219124313,00:28:47,...,19,2020-12-19 12:14:19,2020,12,19,12,43,13,2020-12-19 12:43:13,0 days 00:28:54
2796613,거창휴게소(옥포방향),1,경상남도,거창군,경상남도 거창군 가조면 도리,50.0,DC차데모+AC3상+DC콤보,20201219120725,20201219124405,00:36:34,...,25,2020-12-19 12:07:25,2020,12,19,12,44,5,2020-12-19 12:44:05,0 days 00:36:40


In [699]:
df[df['충전시작_년도'] == 2021].head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
2796614,군위휴게소(부산방향),1,경상북도,군위군,경상북도 군위군 군위읍 경북대로 4084 (오곡리),NaN,DC차데모+AC3상+DC콤보,20210101092250,20210101095816,00:29:24,...,50,2021-01-01 09:22:50,2021,1,1,9,58,16,2021-01-01 09:58:16,0 days 00:35:26
2796615,충주휴게소(마산방향),1,충청북도,충주시,충청북도 충주시 중앙탑면 중부내륙고속도로 231 (용전리),NaN,DC차데모+AC3상+DC콤보,20210101095126,20210101095818,00:06:48,...,26,2021-01-01 09:51:26,2021,1,1,9,58,18,2021-01-01 09:58:18,0 days 00:06:52
2796616,내서읍스포츠센터,1,경상남도,창원시,경상남도 창원시 마산회원구 내서읍 중리 391-3,NaN,DC콤보,20210101094752,20210101095819,00:10:27,...,52,2021-01-01 09:47:52,2021,1,1,9,58,19,2021-01-01 09:58:19,0 days 00:10:27
2796617,삼성혈 인근 공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 이도1동 1293-1,NaN,DC차데모+AC3상+DC콤보,20210101091323,20210101095819,00:40:00,...,23,2021-01-01 09:13:23,2021,1,1,9,58,19,2021-01-01 09:58:19,0 days 00:44:56
2796618,본오1동행정복지센터,1,경기도,안산시,경기도 안산시 상록구 샘골로 16,NaN,DC콤보,20210101091559,20210101095835,00:40:00,...,59,2021-01-01 09:15:59,2021,1,1,9,58,35,2021-01-01 09:58:35,0 days 00:42:36


In [701]:
temp = df[df['충전시작_년도'] == 2020]['충전기용량(kW)'].to_list()
temp[-5:]

[50.0, 50.0, 50.0, 50.0, 50.0]

In [702]:
len(temp)

2796614

In [703]:
temp2 = df[df['충전시작_년도'] == 2021]['충전용량'].to_list()
temp2[-5:]

[50.0, 50.0, 50.0, 50.0, 50.0]

In [704]:
len(temp2)

4259845

In [705]:
temp3 = df[df['충전시작_년도'] == 2022]['충전용량'].to_list()
temp3[-5:]

['급속(50kW)', '급속(200kW동시)', '급속(50kW)', '급속(50kW)', '급속(100kW멀티)']

In [706]:
len(temp3)

4609363

In [707]:
len(temp + temp2 + temp3)

11665822

In [708]:
df['충전용량'] = temp + temp2 + temp3
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,...,31,2020-01-17 19:33:31,2020,1,17,20,14,17,2020-01-17 20:14:17,0 days 00:40:46
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,...,45,2020-01-30 14:35:45,2020,1,30,15,6,45,2020-01-30 15:06:45,0 days 00:31:00
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,...,28,2020-01-05 00:57:28,2020,1,5,1,13,29,2020-01-05 01:13:29,0 days 00:16:01
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,...,15,2020-01-20 12:02:15,2020,1,20,12,47,8,2020-01-20 12:47:08,0 days 00:44:53
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,...,17,2020-01-23 08:54:17,2020,1,23,9,25,54,2020-01-23 09:25:54,0 days 00:31:37


In [709]:
df.iloc[:5, :12]

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,50.0
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,50.0
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,50.0
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,100.0
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,50.0


##### 일시 분리 데이터 출력

In [710]:
df.to_csv('2020-22년_급속충전_일시분리.csv', encoding='utf-8')

##### 일시 분리 데이터 로드(다시 시작)

In [5]:
# df = pd.read_csv('2020-22년_급속충전_일시분리.csv', index_col=0)
# df.info()

C:\Users\kbj\AppData\Local\Temp\ipykernel_7780\927583139.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('2020-22년_급속충전_일시분리.csv')


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 28 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   충전소명        object 
 2   충전기ID       int64  
 3   지역          object 
 4   시군구         object 
 5   주소          object 
 6   충전기용량(kW)   float64
 7   충전기타입       object 
 8   충전시작일시      int64  
 9   충전종료일시      int64  
 10  충전시간        object 
 11  충전량         float64
 12  충전용량        object 
 13  충전시작_년도     int64  
 14  충전시작_월      int64  
 15  충전시작_일      int64  
 16  충전시작_시      int64  
 17  충전시작_분      int64  
 18  충전시작_초      int64  
 19  충전시작일시(변환)  object 
 20  충전종료_년도     int64  
 21  충전종료_월      int64  
 22  충전종료_일      int64  
 23  충전종료_시      int64  
 24  충전종료_분      int64  
 25  충전종료_초      int64  
 26  충전종료일시(변환)  object 
 27  충전시간(계산)    object 
dtypes: float64(2), int64(16), object(10)
memory usage: 2.4+ GB


In [7]:
df = df.drop(['Unnamed: 0'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 27 columns):
 #   Column      Dtype  
---  ------      -----  
 0   충전소명        object 
 1   충전기ID       int64  
 2   지역          object 
 3   시군구         object 
 4   주소          object 
 5   충전기용량(kW)   float64
 6   충전기타입       object 
 7   충전시작일시      int64  
 8   충전종료일시      int64  
 9   충전시간        object 
 10  충전량         float64
 11  충전용량        object 
 12  충전시작_년도     int64  
 13  충전시작_월      int64  
 14  충전시작_일      int64  
 15  충전시작_시      int64  
 16  충전시작_분      int64  
 17  충전시작_초      int64  
 18  충전시작일시(변환)  object 
 19  충전종료_년도     int64  
 20  충전종료_월      int64  
 21  충전종료_일      int64  
 22  충전종료_시      int64  
 23  충전종료_분      int64  
 24  충전종료_초      int64  
 25  충전종료일시(변환)  object 
 26  충전시간(계산)    object 
dtypes: float64(2), int64(15), object(10)
memory usage: 2.3+ GB


### 필요없는 열 제거

In [8]:
# mod_df = df.drop(['충전기용량(kW)', '충전시작_년도', '충전시작_월', '충전시작_일', '충전시작_시', '충전시작_월', '충전시작_일', '충전시작_시', '충전시작_분', '충전시작_초', 
#          '충전종료_년도', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_분', '충전종료_초'], axis=1)
# mod_df.head()

### raw 충전시간과 계산값 비교

In [9]:
df['충전시간'].head()

0    00:40:00
1    00:30:31
2    00:15:35
3    00:41:00
4    00:31:14
Name: 충전시간, dtype: object

In [10]:
df['충전시간'].dtype

dtype('O')

#### 충전시간 오류 값 처리
- 7개 제거

In [11]:
# temp = pd.to_timedelta(df['충전시간']) # only leading negative signs are allowed 에러 발생
temp = pd.to_timedelta(df['충전시간'], errors='coerce')

In [12]:
temp.isnull().sum()

7

In [13]:
temp = temp.to_frame()

In [14]:
temp[temp['충전시간'].isnull() == True]

,충전시간
1207828,NaT
1352351,NaT
1362905,NaT
1372346,NaT
6796417,NaT
7111195,NaT
11392158,NaT


In [15]:
null_idx = temp[temp['충전시간'].isnull() == True].index
for i in null_idx:
    print(df.loc[i, '충전시간'], df.loc[i, '충전시작일시(변환)'])

00:-1:-4 2020-07-06 09:05:47
00:-4:-1 2020-07-05 15:39:18
00:-1:-2 2020-07-06 09:05:47
-6:-2:-5 2020-07-01 19:35:59
00:-4:-2 2021-12-30 17:18:27
00:-1:-3 2022-01-14 17:53:48
-1:-4:-4 2022-08-19 03:00:11


In [16]:
df.shape

(11665822, 27)

In [17]:
df2 = df.drop(null_idx, axis=0)
df2.shape

(11665815, 27)

In [18]:
temp = pd.to_timedelta(df2['충전시간'])
temp.head()

0   0 days 00:40:00
1   0 days 00:30:31
2   0 days 00:15:35
3   0 days 00:41:00
4   0 days 00:31:14
Name: 충전시간, dtype: timedelta64[ns]

In [19]:
df2['충전시간'] = pd.to_timedelta(df2['충전시간'])
df2['충전시간(계산)'] = pd.to_timedelta(df2['충전시간(계산)'])
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11665815 entries, 0 to 11665821
Data columns (total 27 columns):
 #   Column      Dtype          
---  ------      -----          
 0   충전소명        object         
 1   충전기ID       int64          
 2   지역          object         
 3   시군구         object         
 4   주소          object         
 5   충전기용량(kW)   float64        
 6   충전기타입       object         
 7   충전시작일시      int64          
 8   충전종료일시      int64          
 9   충전시간        timedelta64[ns]
 10  충전량         float64        
 11  충전용량        object         
 12  충전시작_년도     int64          
 13  충전시작_월      int64          
 14  충전시작_일      int64          
 15  충전시작_시      int64          
 16  충전시작_분      int64          
 17  충전시작_초      int64          
 18  충전시작일시(변환)  object         
 19  충전종료_년도     int64          
 20  충전종료_월      int64          
 21  충전종료_일      int64          
 22  충전종료_시      int64          
 23  충전종료_분      int64          
 24  충전종료_초      int64     

In [20]:
df2['충전시간'].describe()

count                     11665815
mean     0 days 00:34:21.381945453
std      0 days 01:20:05.913477577
min                0 days 00:00:00
25%                0 days 00:20:55
50%                0 days 00:34:07
75%                0 days 00:40:00
max                4 days 03:56:55
Name: 충전시간, dtype: object

In [21]:
df2['충전시간(계산)'].describe()

count                     11665815
mean     0 days 00:29:18.506520633
std      4 days 08:44:42.009406092
min          -14905 days +09:10:08
25%                0 days 00:21:28
50%                0 days 00:34:49
75%                0 days 00:40:24
max               14 days 08:40:25
Name: 충전시간(계산), dtype: object

In [22]:
df2[df2['충전시간(계산)'] == '-14905 days +09:10:08']

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
2120856,단양관광안내소(단양팔경),1,충청북도,단양군,충청북도 단양군 단양읍 고수리 149-1,50.0,DC차데모+AC3상+DC콤보,20201030144953,19800110000001,3 days 10:47:44,...,53,2020-10-30 14:49:53,1980,1,10,0,0,1,1980-01-10 00:00:01,-14905 days +09:10:08


#### 충전시간 데이터 확인

In [23]:
df2[df2['충전시간'] == '0 days 00:00:00'].head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
31211,(재)문화엑스포,11,경상북도,경주시,경상북도 경주시 경감로 614,200.0,DC콤보,20200109145420,20200109145420,0 days,...,20,2020-01-09 14:54:20,2020,1,9,14,54,20,2020-01-09 14:54:20,0 days 00:00:00
61525,종합사회복지관,12,충청남도,공주시,충청남도 공주시 의당면 의당로 257,200.0,DC콤보,20200114192124,20200114192132,0 days,...,24,2020-01-14 19:21:24,2020,1,14,19,21,32,2020-01-14 19:21:32,0 days 00:00:08
96952,해남군청 주차장,12,전라남도,해남군,전라남도 해남군 해남읍 수성리 183-5,200.0,DC콤보,20200124160600,20200124160601,0 days,...,0,2020-01-24 16:06:00,2020,1,24,16,6,1,2020-01-24 16:06:01,0 days 00:00:01
118208,롯데 빅마켓,3,서울특별시,영등포구,"서울특별시 영등포구 영중로 125, 지상주차장(외부 1F)",100.0,DC콤보,20200122012647,20200122012716,0 days,...,47,2020-01-22 01:26:47,2020,1,22,1,27,16,2020-01-22 01:27:16,0 days 00:00:29
174295,이마트 파주운정점,11,경기도,파주시,경기도 파주시 한울로 123,200.0,DC콤보,20200107165228,20200107165255,0 days,...,28,2020-01-07 16:52:28,2020,1,7,16,52,55,2020-01-07 16:52:55,0 days 00:00:27


In [24]:
df2[df2['충전시간'] == '0 days 00:00:00'].shape

(5702, 27)

In [25]:
df2['충전량'].dtype

dtype('float64')

In [26]:
df2['충전량'].head()

0    25.47
1    17.57
2     6.98
3    15.30
4    21.92
Name: 충전량, dtype: float64

In [27]:
df2[ (df2['충전시간'] == '0 days 00:00:00') & (df2['충전량'] != 0) ][['충전시간', '충전량']]

,충전시간,충전량
31211,0 days,1.18
61525,0 days,14.58
96952,0 days,13.33
118208,0 days,15.79
174295,0 days,22.51
...,...,...
11571988,0 days,18.70
11572694,0 days,23.41
11607868,0 days,7.44
11618047,0 days,4.50


In [28]:
df2[ (df2['충전시간'] == '0 days 00:00:00') & (df2['충전량'] != 0) ].shape

(5702, 27)

#### 충전시간(계산) 음수 값 제거
- 52366개

In [29]:
n_ch_time_idx = df2[df2['충전시간(계산)'] < '0'].index
len(n_ch_time_idx)

52366

In [30]:
df2 = df2.drop(n_ch_time_idx)

In [31]:
df2['충전시간'].describe()

count                     11613449
mean     0 days 00:33:51.200344617
std      0 days 01:19:04.781739278
min                0 days 00:00:00
25%                0 days 00:20:53
50%                0 days 00:34:02
75%                0 days 00:40:00
max                4 days 03:56:55
Name: 충전시간, dtype: object

In [32]:
df2['충전시간(계산)'].describe()

count                     11613449
mean     0 days 00:34:58.600275335
std      0 days 01:23:19.668917229
min                0 days 00:00:00
25%                0 days 00:21:39
50%                0 days 00:34:56
75%                0 days 00:40:25
max               14 days 08:40:25
Name: 충전시간(계산), dtype: object

In [33]:
df2['충전시간(계산)'].astype(str)[:5]

0    0 days 00:40:46
1    0 days 00:31:00
2    0 days 00:16:01
3    0 days 00:44:53
4    0 days 00:31:37
Name: 충전시간(계산), dtype: object

In [34]:
df2['충전시간'][:5]

0   0 days 00:40:00
1   0 days 00:30:31
2   0 days 00:15:35
3   0 days 00:41:00
4   0 days 00:31:14
Name: 충전시간, dtype: timedelta64[ns]

In [35]:
df2['충전시간'].dt.seconds[:5]

0    2400
1    1831
2     935
3    2460
4    1874
Name: 충전시간, dtype: int64

In [36]:
temp = df2['충전시간(계산)'] - df2['충전시간']
temp.describe()

count                     11613449
mean     0 days 00:01:07.399930718
std      0 days 01:01:06.908441070
min              -1 days +00:00:23
25%                0 days 00:00:00
50%                0 days 00:00:13
75%                0 days 00:01:42
max               14 days 08:32:12
dtype: object

In [37]:
temp[temp < '0'].shape

(817340,)

In [38]:
df2['raw값과_계산값_차이'] = df2['충전시간(계산)'] - df2['충전시간']

In [39]:
df2[df2['raw값과_계산값_차이'] > '14 days']

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산),raw값과_계산값_차이
8510493,포항실내사격장,11,경상북도,포항시,경상북도 포항시 북구 용흥동 4-9,NaN,DC콤보,20220305094551,20220319182616,0 days 00:08:13,...,2022-03-05 09:45:51,2022,3,19,18,26,16,2022-03-19 18:26:16,14 days 08:40:25,14 days 08:32:12


In [40]:
# df2 = df2.drop(['raw값과_계산값_차이'], axis=1)

In [41]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 28 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int64          
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기용량(kW)     float64        
 6   충전기타입         object         
 7   충전시작일시        int64          
 8   충전종료일시        int64          
 9   충전시간          timedelta64[ns]
 10  충전량           float64        
 11  충전용량          object         
 12  충전시작_년도       int64          
 13  충전시작_월        int64          
 14  충전시작_일        int64          
 15  충전시작_시        int64          
 16  충전시작_분        int64          
 17  충전시작_초        int64          
 18  충전시작일시(변환)    object         
 19  충전종료_년도       int64          
 20  충전종료_월        int64          
 21  충전종료_일        int64          
 22  충전종료_시        int64          
 23  충전종료_

In [44]:
df2.iloc[:, :14].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 14 columns):
 #   Column     Dtype          
---  ------     -----          
 0   충전소명       object         
 1   충전기ID      int64          
 2   지역         object         
 3   시군구        object         
 4   주소         object         
 5   충전기용량(kW)  float64        
 6   충전기타입      object         
 7   충전시작일시     int64          
 8   충전종료일시     int64          
 9   충전시간       timedelta64[ns]
 10  충전량        float64        
 11  충전용량       object         
 12  충전시작_년도    int64          
 13  충전시작_월     int64          
dtypes: float64(2), int64(5), object(6), timedelta64[ns](1)
memory usage: 1.3+ GB


In [45]:
df2.iloc[:, 14:].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 14 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전시작_일        int64          
 1   충전시작_시        int64          
 2   충전시작_분        int64          
 3   충전시작_초        int64          
 4   충전시작일시(변환)    object         
 5   충전종료_년도       int64          
 6   충전종료_월        int64          
 7   충전종료_일        int64          
 8   충전종료_시        int64          
 9   충전종료_분        int64          
 10  충전종료_초        int64          
 11  충전종료일시(변환)    object         
 12  충전시간(계산)      timedelta64[ns]
 13  raw값과_계산값_차이  timedelta64[ns]
dtypes: int64(10), object(2), timedelta64[ns](2)
memory usage: 1.3+ GB


### 데이터 절약

In [47]:
pd.options.display.float_format = '{:.5f}'.format

In [49]:
time_cols = ['충전시작_년도', '충전시작_월',  '충전시작_일', '충전시작_시', '충전시작_분', '충전시작_초', '충전종료_년도', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_분', '충전종료_초']
df2[time_cols].describe()

,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초
count,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000
mean,2021.16059,6.20425,15.80842,13.72640,29.47740,29.49963,2021.16062,6.20430,15.80841,14.01221,29.54277,29.45744
std,0.77937,3.33015,8.80106,4.95829,17.33414,17.31996,0.77937,3.33017,8.80110,5.07576,17.34527,17.33907
min,2020.00000,1.00000,1.00000,0.00000,0.00000,0.00000,2020.00000,1.00000,1.00000,0.00000,0.00000,0.00000
25%,2021.00000,3.00000,8.00000,10.00000,14.00000,14.00000,2021.00000,3.00000,8.00000,11.00000,15.00000,14.00000
50%,2021.00000,6.00000,16.00000,14.00000,29.00000,29.00000,2021.00000,6.00000,16.00000,14.00000,30.00000,29.00000
75%,2022.00000,9.00000,23.00000,17.00000,45.00000,45.00000,2022.00000,9.00000,23.00000,18.00000,45.00000,44.00000
max,2022.00000,12.00000,31.00000,23.00000,59.00000,59.00000,2022.00000,12.00000,31.00000,23.00000,59.00000,59.00000


In [53]:
df2[['충전시작_년도', '충전종료_년도']] = df2[['충전시작_년도', '충전종료_년도']].astype('int16')

In [54]:
df2[['충전시작_년도', '충전종료_년도']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   충전시작_년도  int16
 1   충전종료_년도  int16
dtypes: int16(2)
memory usage: 132.9 MB


In [55]:
df2[['충전시작_월',  '충전시작_일', '충전시작_시', '충전시작_분', '충전시작_초', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_분', '충전종료_초']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 10 columns):
 #   Column  Dtype
---  ------  -----
 0   충전시작_월  int64
 1   충전시작_일  int64
 2   충전시작_시  int64
 3   충전시작_분  int64
 4   충전시작_초  int64
 5   충전종료_월  int64
 6   충전종료_일  int64
 7   충전종료_시  int64
 8   충전종료_분  int64
 9   충전종료_초  int64
dtypes: int64(10)
memory usage: 974.6 MB


In [57]:
df2[['충전시작_월',  '충전시작_일', '충전시작_시', '충전시작_분', '충전시작_초', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_분', '충전종료_초']] = df2[['충전시작_월',  '충전시작_일', '충전시작_시', '충전시작_분', '충전시작_초', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_분', '충전종료_초']].astype('int8')
df2[['충전시작_월',  '충전시작_일', '충전시작_시', '충전시작_분', '충전시작_초', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_분', '충전종료_초']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 10 columns):
 #   Column  Dtype
---  ------  -----
 0   충전시작_월  int8 
 1   충전시작_일  int8 
 2   충전시작_시  int8 
 3   충전시작_분  int8 
 4   충전시작_초  int8 
 5   충전종료_월  int8 
 6   충전종료_일  int8 
 7   충전종료_시  int8 
 8   충전종료_분  int8 
 9   충전종료_초  int8 
dtypes: int8(10)
memory usage: 199.4 MB


In [59]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 28 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int64          
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기용량(kW)     float64        
 6   충전기타입         object         
 7   충전시작일시        int64          
 8   충전종료일시        int64          
 9   충전시간          timedelta64[ns]
 10  충전량           float64        
 11  충전용량          object         
 12  충전시작_년도       int16          
 13  충전시작_월        int8           
 14  충전시작_일        int8           
 15  충전시작_시        int8           
 16  충전시작_분        int8           
 17  충전시작_초        int8           
 18  충전시작일시(변환)    object         
 19  충전종료_년도       int16          
 20  충전종료_월        int8           
 21  충전종료_일        int8           
 22  충전종료_시        int8           
 23  충전종료_

In [61]:
df2['충전기ID'].describe()

count   11613449.00000
mean           8.59347
std           15.08407
min            1.00000
25%            1.00000
50%            1.00000
75%           12.00000
max           98.00000
Name: 충전기ID, dtype: float64

In [62]:
df2['충전기ID'].info()

<class 'pandas.core.series.Series'>
Int64Index: 11613449 entries, 0 to 11665821
Series name: 충전기ID
Non-Null Count     Dtype
--------------     -----
11613449 non-null  int64
dtypes: int64(1)
memory usage: 177.2 MB


In [63]:
df2['충전기ID'] = df2['충전기ID'].astype('int16')
df2['충전기ID'].info()

<class 'pandas.core.series.Series'>
Int64Index: 11613449 entries, 0 to 11665821
Series name: 충전기ID
Non-Null Count     Dtype
--------------     -----
11613449 non-null  int16
dtypes: int16(1)
memory usage: 110.8 MB


In [64]:
df2['충전시간'].describe()

count                     11613449
mean     0 days 00:33:51.200344617
std      0 days 01:19:04.781739278
min                0 days 00:00:00
25%                0 days 00:20:53
50%                0 days 00:34:02
75%                0 days 00:40:00
max                4 days 03:56:55
Name: 충전시간, dtype: object

In [65]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 28 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int16          
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기용량(kW)     float64        
 6   충전기타입         object         
 7   충전시작일시        int64          
 8   충전종료일시        int64          
 9   충전시간          timedelta64[ns]
 10  충전량           float64        
 11  충전용량          object         
 12  충전시작_년도       int16          
 13  충전시작_월        int8           
 14  충전시작_일        int8           
 15  충전시작_시        int8           
 16  충전시작_분        int8           
 17  충전시작_초        int8           
 18  충전시작일시(변환)    object         
 19  충전종료_년도       int16          
 20  충전종료_월        int8           
 21  충전종료_일        int8           
 22  충전종료_시        int8           
 23  충전종료_

In [73]:
df2 = df2.drop(['충전기용량(kW)'], axis=1)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 27 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int16          
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_

#### 충전시간 정리 파일로 출력

In [90]:
df2.to_csv('2020-22년_급속충전-충전시간정리.csv', encoding='utf-8')

In [93]:
# df2.to_excel('2020-22년_급속충전-충전시간정리.xlsx', encoding='utf-8') # error : 파일 너무 커서 저장 실패

##### 충전시간 정리 파일 로드(다시 시작)

In [ ]:
# df2 = pd.read_csv('2020-22년_급속충전-충전시간정리.csv', index_col=0)
# df2.info()

### 지역별 충전시간, 충전량 통계
- 총 충전시간, 총 충전량

In [289]:
ch_time_day_by_local = df2.groupby(['지역'])['충전시간'].sum()
ch_time_day_by_local

지역
강원도       19394 days 08:15:45
경기도       44943 days 22:18:50
경상남도      19393 days 17:46:57
경상북도      32622 days 19:07:20
광주광역시      5026 days 00:52:54
대구광역시      8859 days 12:04:57
대전광역시      7934 days 13:38:40
부산광역시      6641 days 06:56:50
서울특별시     20686 days 09:17:13
세종특별자치시    1547 days 10:32:37
울산광역시      5049 days 12:17:23
인천광역시     11199 days 13:48:21
전라남도      15529 days 21:19:46
전라북도      16610 days 18:03:36
제주특별자치도   16986 days 01:29:37
충청남도      21601 days 11:59:59
충청북도      18996 days 07:16:06
Name: 충전시간, dtype: timedelta64[ns]

In [290]:
ch_time_cal_day_by_local = ch_time_day_by_local.to_frame()
ch_time_cal_day_by_local

,충전시간
지역,
강원도,19394 days 08:15:45
경기도,44943 days 22:18:50
경상남도,19393 days 17:46:57
경상북도,32622 days 19:07:20
광주광역시,5026 days 00:52:54
대구광역시,8859 days 12:04:57
대전광역시,7934 days 13:38:40
부산광역시,6641 days 06:56:50
서울특별시,20686 days 09:17:13


In [291]:
temp1 = ch_time_cal_day_by_local['충전시간'].dt.seconds.sum()
temp1

745611

In [292]:
temp2 = ch_time_cal_day_by_local['충전시간'].dt.days.sum()
temp2

273015

In [293]:
type(temp1)

numpy.int64

In [294]:
ch_time_cal_day_by_local['충전시간'] = ch_time_cal_day_by_local['충전시간'].astype(str)
ch_time_cal_day_by_local = ch_time_cal_day_by_local['충전시간'].str.split(expand=True)
ch_time_cal_day_by_local

,0,1,2
지역,,,
강원도,19394,days,08:15:45
경기도,44943,days,22:18:50
경상남도,19393,days,17:46:57
경상북도,32622,days,19:07:20
광주광역시,5026,days,00:52:54
대구광역시,8859,days,12:04:57
대전광역시,7934,days,13:38:40
부산광역시,6641,days,06:56:50
서울특별시,20686,days,09:17:13


In [295]:
months = temp2 // 30
days = temp2 % 30
hours = temp1 // 3600
minutes = (temp1 % 3600) // 60
seconds = (temp1 % 3600) % 60
months, days, hours, minutes, seconds

(9100, 15, 207, 6, 51)

In [296]:
days += hours // 24
hours = hours % 24
days, hours

(23, 15)

In [297]:
ch_time_cal_day_by_local = ch_time_cal_day_by_local.drop([1], axis=1)
ch_time_cal_day_by_local

,0,2
지역,,
강원도,19394,08:15:45
경기도,44943,22:18:50
경상남도,19393,17:46:57
경상북도,32622,19:07:20
광주광역시,5026,00:52:54
대구광역시,8859,12:04:57
대전광역시,7934,13:38:40
부산광역시,6641,06:56:50
서울특별시,20686,09:17:13


In [298]:
ch_time_cal_day_by_local.columns = ['days', 'time']
ch_time_cal_day_by_local

,days,time
지역,,
강원도,19394,08:15:45
경기도,44943,22:18:50
경상남도,19393,17:46:57
경상북도,32622,19:07:20
광주광역시,5026,00:52:54
대구광역시,8859,12:04:57
대전광역시,7934,13:38:40
부산광역시,6641,06:56:50
서울특별시,20686,09:17:13


In [299]:
ch_time_cal_day_by_local['days'] = ch_time_cal_day_by_local['days'].astype(int)

In [300]:
ch_time_cal_day_by_local['months'] = ch_time_cal_day_by_local['days'] // 30
ch_time_cal_day_by_local['days'] = ch_time_cal_day_by_local['days'] % 30
ch_time_cal_day_by_local

,days,time,months
지역,,,
강원도,14,08:15:45,646
경기도,3,22:18:50,1498
경상남도,13,17:46:57,646
경상북도,12,19:07:20,1087
광주광역시,16,00:52:54,167
대구광역시,9,12:04:57,295
대전광역시,14,13:38:40,264
부산광역시,11,06:56:50,221
서울특별시,16,09:17:13,689


In [301]:
ch_time_cal_day_by_local = ch_time_cal_day_by_local[['months', 'days', 'time']]
ch_time_cal_day_by_local

,months,days,time
지역,,,
강원도,646,14,08:15:45
경기도,1498,3,22:18:50
경상남도,646,13,17:46:57
경상북도,1087,12,19:07:20
광주광역시,167,16,00:52:54
대구광역시,295,9,12:04:57
대전광역시,264,14,13:38:40
부산광역시,221,11,06:56:50
서울특별시,689,16,09:17:13


In [302]:
ch_time_cal_day_by_local.shape

(17, 3)

In [303]:
ch_time_cal_day_by_local.columns

Index(['months', 'days', 'time'], dtype='object')

In [304]:
temp = pd.DataFrame({'months':[months], 'days':[days], 'time':[f'{hours}:{minutes}:{seconds}']})
temp

,months,days,time
0,9100,23,15:6:51


In [305]:
ch_time_cal_day_by_local = pd.concat([ch_time_cal_day_by_local, temp])
ch_time_cal_day_by_local

,months,days,time
강원도,646,14,08:15:45
경기도,1498,3,22:18:50
경상남도,646,13,17:46:57
경상북도,1087,12,19:07:20
광주광역시,167,16,00:52:54
대구광역시,295,9,12:04:57
대전광역시,264,14,13:38:40
부산광역시,221,11,06:56:50
서울특별시,689,16,09:17:13
세종특별자치시,51,17,10:32:37


In [311]:
ch_time_cal_day_by_local.columns = [['지역별 총 충전시간']*3, ['months', 'days', 'time']]
ch_time_cal_day_by_local

지역별 총 충전시간               
            months days      time
강원도            646   14  08:15:45
경기도           1498    3  22:18:50
경상남도           646   13  17:46:57
경상북도          1087   12  19:07:20
광주광역시          167   16  00:52:54
대구광역시          295    9  12:04:57
대전광역시          264   14  13:38:40
부산광역시          221   11  06:56:50
서울특별시          689   16  09:17:13
세종특별자치시         51   17  10:32:37
울산광역시          168    9  12:17:23
인천광역시          373    9  13:48:21
전라남도           517   19  21:19:46
전라북도           553   20  18:03:36
제주특별자치도        566    6  01:29:37
충청남도           720    1  11:59:59
충청북도           633    6  07:16:06
0             9100   23   15:6:51

In [312]:
ch_time_cal_day_by_local.to_excel('2020-22년_급속충전_지역별_총_충전시간.xlsx', encoding='utf-8')

#### 총 충전시간(계산)

In [327]:
col = '충전시간(계산)'
ch_time_by_local = df2.groupby(['지역'])[col].sum()
ch_time_by_local

지역
강원도       20379 days 13:42:02
경기도       46362 days 07:47:30
경상남도      20375 days 03:50:30
경상북도      34057 days 10:27:05
광주광역시      5125 days 22:19:49
대구광역시      9433 days 15:16:10
대전광역시      8068 days 03:05:12
부산광역시      6848 days 11:00:24
서울특별시     21362 days 05:17:37
세종특별자치시    1607 days 02:52:01
울산광역시      5269 days 06:06:50
인천광역시     11007 days 01:44:22
전라남도      16178 days 02:20:27
전라북도      16372 days 14:46:27
제주특별자치도   17992 days 11:34:50
충청남도      22417 days 11:17:12
충청북도      19226 days 04:59:21
Name: 충전시간(계산), dtype: timedelta64[ns]

In [328]:
temp = ch_time_by_local.to_frame()
temp

,충전시간(계산)
지역,
강원도,20379 days 13:42:02
경기도,46362 days 07:47:30
경상남도,20375 days 03:50:30
경상북도,34057 days 10:27:05
광주광역시,5125 days 22:19:49
대구광역시,9433 days 15:16:10
대전광역시,8068 days 03:05:12
부산광역시,6848 days 11:00:24
서울특별시,21362 days 05:17:37


In [329]:
temp1 = temp[col].dt.seconds.sum()
temp1

534469

In [330]:
temp2 = temp[col].dt.days.sum()
temp2

282077

In [331]:
temp[col] = temp[col].astype(str)
temp = temp[col].str.split(expand=True)
temp

,0,1,2
지역,,,
강원도,20379,days,13:42:02
경기도,46362,days,07:47:30
경상남도,20375,days,03:50:30
경상북도,34057,days,10:27:05
광주광역시,5125,days,22:19:49
대구광역시,9433,days,15:16:10
대전광역시,8068,days,03:05:12
부산광역시,6848,days,11:00:24
서울특별시,21362,days,05:17:37


In [332]:
months = temp2 // 30
days = temp2 % 30
hours = temp1 // 3600
minutes = (temp1 % 3600) // 60
seconds = (temp1 % 3600) % 60
months, days, hours, minutes, seconds

(9402, 17, 148, 27, 49)

In [333]:
days += hours // 24
hours = hours % 24
days, hours

(23, 4)

In [334]:
temp = temp.drop([1], axis=1)
temp

,0,2
지역,,
강원도,20379,13:42:02
경기도,46362,07:47:30
경상남도,20375,03:50:30
경상북도,34057,10:27:05
광주광역시,5125,22:19:49
대구광역시,9433,15:16:10
대전광역시,8068,03:05:12
부산광역시,6848,11:00:24
서울특별시,21362,05:17:37


In [335]:
temp.columns = ['days', 'time']
temp

,days,time
지역,,
강원도,20379,13:42:02
경기도,46362,07:47:30
경상남도,20375,03:50:30
경상북도,34057,10:27:05
광주광역시,5125,22:19:49
대구광역시,9433,15:16:10
대전광역시,8068,03:05:12
부산광역시,6848,11:00:24
서울특별시,21362,05:17:37


In [336]:
temp['days'] = temp['days'].astype(int)

In [337]:
temp['months'] = temp['days'] // 30
temp['days'] = temp['days'] % 30
temp

,days,time,months
지역,,,
강원도,9,13:42:02,679
경기도,12,07:47:30,1545
경상남도,5,03:50:30,679
경상북도,7,10:27:05,1135
광주광역시,25,22:19:49,170
대구광역시,13,15:16:10,314
대전광역시,28,03:05:12,268
부산광역시,8,11:00:24,228
서울특별시,2,05:17:37,712


In [338]:
temp = temp[['months', 'days', 'time']]
temp

,months,days,time
지역,,,
강원도,679,9,13:42:02
경기도,1545,12,07:47:30
경상남도,679,5,03:50:30
경상북도,1135,7,10:27:05
광주광역시,170,25,22:19:49
대구광역시,314,13,15:16:10
대전광역시,268,28,03:05:12
부산광역시,228,8,11:00:24
서울특별시,712,2,05:17:37


In [339]:
temp.columns

Index(['months', 'days', 'time'], dtype='object')

In [340]:
total = pd.DataFrame({'months':[months], 'days':[days], 'time':[f'{hours}:{minutes}:{seconds}']})
total

,months,days,time
0,9402,23,4:27:49


In [341]:
temp = pd.concat([temp, total])
temp

,months,days,time
강원도,679,9,13:42:02
경기도,1545,12,07:47:30
경상남도,679,5,03:50:30
경상북도,1135,7,10:27:05
광주광역시,170,25,22:19:49
대구광역시,314,13,15:16:10
대전광역시,268,28,03:05:12
부산광역시,228,8,11:00:24
서울특별시,712,2,05:17:37
세종특별자치시,53,17,02:52:01


In [342]:
temp.columns = [['지역별 총 충전시간(계산)']*3, ['months', 'days', 'time']]
temp

지역별 총 충전시간(계산)               
                months days      time
강원도                679    9  13:42:02
경기도               1545   12  07:47:30
경상남도               679    5  03:50:30
경상북도              1135    7  10:27:05
광주광역시              170   25  22:19:49
대구광역시              314   13  15:16:10
대전광역시              268   28  03:05:12
부산광역시              228    8  11:00:24
서울특별시              712    2  05:17:37
세종특별자치시             53   17  02:52:01
울산광역시              175   19  06:06:50
인천광역시              366   27  01:44:22
전라남도               539    8  02:20:27
전라북도               545   22  14:46:27
제주특별자치도            599   22  11:34:50
충청남도               747    7  11:17:12
충청북도               640   26  04:59:21
0                 9402   23   4:27:49

In [343]:
temp.to_excel('2020-22년_급속충전_지역별_총_충전시간(계산).xlsx', encoding='utf-8')

#### 충전량

In [147]:
ch_vol_by_local = df2.groupby(['지역'])['충전량'].sum()
ch_vol_by_local

지역
강원도        16232572.48
경기도        37043650.51
경상남도       17777425.02
경상북도       27188951.50
광주광역시       3935268.34
대구광역시       7842277.45
대전광역시       6176372.79
부산광역시       5405663.34
서울특별시      16046052.85
세종특별자치시     1298904.64
울산광역시       4329300.12
인천광역시       8821079.64
전라남도       12601306.22
전라북도       14003630.09
제주특별자치도    12207723.80
충청남도       18808718.07
충청북도       15520247.58
Name: 충전량, dtype: float64

In [33]:
ch_vol_by_local.to_excel('2020-22년_급속충전_지역별_총_충전량.xlsx', encoding='utf-8')

### 년도별 충전시간, 충전량 통계
- 총 충전시간, 충전량

In [71]:
ch_time_day_by_year = df2.groupby(['충전시작_년도'])['충전시간'].sum()
ch_time_day_by_year

충전시작_년도
2020             89296 days 01:56:46
2021             88766 days 15:01:20
2022   94960 days 22:08:44.999999488
Name: 충전시간, dtype: timedelta64[ns]

In [72]:
ch_time_day_by_year.to_excel('2020-22년_급속충전_년도별_총_충전시간.xlsx', encoding='utf-8')

### 년도, 월별 충전시간, 충전량 통계
- 총 충전시간, 총 충전량

#### 총 충전시간

In [568]:
col = '충전시간'
ch_cal_time_by_ym = df2.groupby(['충전시작_년도', '충전시작_월'])[col].sum()
ch_cal_time_by_ym.head()

충전시작_년도  충전시작_월
2020     1        4844 days 06:04:49
         2        3807 days 14:53:40
         3        4220 days 18:42:13
         4        5200 days 19:02:23
         5        6698 days 16:05:39
Name: 충전시간, dtype: timedelta64[ns]

In [569]:
ch_cal_time_by_ym = ch_cal_time_by_ym.to_frame()
ch_cal_time_by_ym.head()

충전시간
충전시작_년도 충전시작_월                   
2020    1      4844 days 06:04:49
        2      3807 days 14:53:40
        3      4220 days 18:42:13
        4      5200 days 19:02:23
        5      6698 days 16:05:39

In [570]:
temp1 = ch_cal_time_by_ym[col].dt.seconds.sum()
temp1

1264011

In [571]:
temp2 = ch_cal_time_by_ym[col].dt.days.sum()
temp2

273009

In [572]:
months = temp2 // 30
days = temp2 % 30
hours = temp1 // 3600
minutes = (temp1 % 3600) // 60
seconds = (temp1 % 3600) % 60
months, days, hours, minutes, seconds

(9100, 9, 351, 6, 51)

In [573]:
days += hours // 24
hours = hours % 24
days, hours

(23, 15)

In [574]:
ch_cal_time_by_ym[col] = ch_cal_time_by_ym[col].astype(str)
temp = ch_cal_time_by_ym[col].str.split(expand=True)
temp

0     1         2
충전시작_년도 충전시작_월                       
2020    1        4844  days  06:04:49
        2        3807  days  14:53:40
        3        4220  days  18:42:13
        4        5200  days  19:02:23
        5        6698  days  16:05:39
        6        8425  days  07:42:45
        7        9948  days  17:47:42
        8        9994  days  13:12:00
        9        7887  days  09:02:08
        10       9809  days  02:56:50
        11       9686  days  05:46:34
        12       8772  days  14:40:03
2021    1        6700  days  00:31:35
        2        5972  days  01:10:01
        3        6318  days  03:25:10
        4        5981  days  18:18:29
        5        5934  days  01:30:44
        6        5927  days  10:26:42
        7        6999  days  08:35:22
        8        7217  days  06:09:09
        9        7222  days  08:48:45
        10       8611  days  17:43:58
        11      10017  days  21:16:40
        12      11864  days  13:04:45
2022    1       11766  days  19:03:00
        2        9909  days  20:56:47
        3       10833  days  12:08:33
        4       11210  days  01:08:59
        5       12025  days  11:31:03
        6       12032  days  13:22:44
        7       13605  days  05:53:35
        8       13577  days  10:04:04

In [575]:
temp = temp.drop([1], axis=1)
temp.columns = ['days', 'time']
temp['months'] = temp['days'].astype(int) // 30
temp['days'] = temp['days'].astype(int) % 30
temp

days      time  months
충전시작_년도 충전시작_월                        
2020    1         14  06:04:49     161
        2         27  14:53:40     126
        3         20  18:42:13     140
        4         10  19:02:23     173
        5          8  16:05:39     223
        6         25  07:42:45     280
        7         18  17:47:42     331
        8          4  13:12:00     333
        9         27  09:02:08     262
        10        29  02:56:50     326
        11        26  05:46:34     322
        12        12  14:40:03     292
2021    1         10  00:31:35     223
        2          2  01:10:01     199
        3         18  03:25:10     210
        4         11  18:18:29     199
        5         24  01:30:44     197
        6         17  10:26:42     197
        7          9  08:35:22     233
        8         17  06:09:09     240
        9         22  08:48:45     240
        10         1  17:43:58     287
        11        27  21:16:40     333
        12        14  13:04:45     395
2022    1          6  19:03:00     392
        2          9  20:56:47     330
        3          3  12:08:33     361
        4         20  01:08:59     373
        5         25  11:31:03     400
        6          2  13:22:44     401
        7         15  05:53:35     453
        8         17  10:04:04     452

In [576]:
temp.columns

Index(['days', 'time', 'months'], dtype='object')

In [577]:
sp1 = temp.iloc[:12, :]
sp1

days      time  months
충전시작_년도 충전시작_월                        
2020    1         14  06:04:49     161
        2         27  14:53:40     126
        3         20  18:42:13     140
        4         10  19:02:23     173
        5          8  16:05:39     223
        6         25  07:42:45     280
        7         18  17:47:42     331
        8          4  13:12:00     333
        9         27  09:02:08     262
        10        29  02:56:50     326
        11        26  05:46:34     322
        12        12  14:40:03     292

In [578]:
sp2 = temp.iloc[12:24, :]
sp2

days      time  months
충전시작_년도 충전시작_월                        
2021    1         10  00:31:35     223
        2          2  01:10:01     199
        3         18  03:25:10     210
        4         11  18:18:29     199
        5         24  01:30:44     197
        6         17  10:26:42     197
        7          9  08:35:22     233
        8         17  06:09:09     240
        9         22  08:48:45     240
        10         1  17:43:58     287
        11        27  21:16:40     333
        12        14  13:04:45     395

In [579]:
sp3 = temp.iloc[24:, :]
sp3

days      time  months
충전시작_년도 충전시작_월                        
2022    1          6  19:03:00     392
        2          9  20:56:47     330
        3          3  12:08:33     361
        4         20  01:08:59     373
        5         25  11:31:03     400
        6          2  13:22:44     401
        7         15  05:53:35     453
        8         17  10:04:04     452

In [580]:
ch_cal_time_by_y = df2.groupby(['충전시작_년도'])[col].sum()
ch_cal_time_by_y.head()

충전시작_년도
2020             89296 days 01:56:46
2021             88766 days 15:01:20
2022   94960 days 22:08:44.999999488
Name: 충전시간, dtype: timedelta64[ns]

In [581]:
ch_cal_time_by_y = ch_cal_time_by_y.to_frame()
ch_cal_time_by_y[col] = ch_cal_time_by_y[col].astype(str)
ch_cal_time_by_y

,충전시간
충전시작_년도,
2020,89296 days 01:56:46
2021,88766 days 15:01:20
2022,94960 days 22:08:44.999999488


In [582]:
ch_cal_time_by_y = ch_cal_time_by_y[col].str.split(expand=True)
ch_cal_time_by_y

,0,1,2
충전시작_년도,,,
2020,89296,days,01:56:46
2021,88766,days,15:01:20
2022,94960,days,22:08:44.999999488


In [583]:
ch_cal_time_by_y = ch_cal_time_by_y.drop([1], axis=1)
ch_cal_time_by_y

,0,2
충전시작_년도,,
2020,89296,01:56:46
2021,88766,15:01:20
2022,94960,22:08:44.999999488


In [584]:
ch_cal_time_by_y[0] = ch_cal_time_by_y[0].astype(int)
ch_cal_time_by_y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 2020 to 2022
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       3 non-null      int32 
 1   2       3 non-null      object
dtypes: int32(1), object(1)
memory usage: 60.0+ bytes


In [585]:
temp20 = ch_cal_time_by_y.iloc[0]
temp21 = ch_cal_time_by_y.iloc[1]
temp22 = ch_cal_time_by_y.iloc[2]

In [586]:
d20, t20 = temp20[0], temp20[2]
months20 = d20 // 30
days20 = d20 % 30
months20, days20, t20

(2976, 16, '01:56:46')

In [587]:
d21, t21 = temp21[0], temp21[2]
months21 = d21 // 30
days21 = d21 % 30
months21, days21, t21

(2958, 26, '15:01:20')

In [588]:
d22,t22 = temp22[0], temp22[2]
months22 = d22 // 30
days22 = d22 % 30
months22, days22, t22

(3165, 10, '22:08:44.999999488')

In [589]:
sp1.columns

Index(['days', 'time', 'months'], dtype='object')

In [590]:
total20 = pd.DataFrame({'days':[days20], 'time':[t20], 'months':[months20]})
total20

,days,time,months
0,16,01:56:46,2976


In [591]:
total21 = pd.DataFrame({'days':[days21], 'time':[t21], 'months':[months21]})
total21

,days,time,months
0,26,15:01:20,2958


In [592]:
total22 = pd.DataFrame({'days':[days22], 'time':[t22], 'months':[months22]})
total22

,days,time,months
0,10,22:08:44.999999488,3165


In [593]:
temp20 = pd.concat([sp1, total20])
temp20

,days,time,months
"(2020, 1)",14,06:04:49,161
"(2020, 2)",27,14:53:40,126
"(2020, 3)",20,18:42:13,140
"(2020, 4)",10,19:02:23,173
"(2020, 5)",8,16:05:39,223
"(2020, 6)",25,07:42:45,280
"(2020, 7)",18,17:47:42,331
"(2020, 8)",4,13:12:00,333
"(2020, 9)",27,09:02:08,262
"(2020, 10)",29,02:56:50,326


In [594]:
temp21 = pd.concat([sp2, total21])
temp21

,days,time,months
"(2021, 1)",10,00:31:35,223
"(2021, 2)",2,01:10:01,199
"(2021, 3)",18,03:25:10,210
"(2021, 4)",11,18:18:29,199
"(2021, 5)",24,01:30:44,197
"(2021, 6)",17,10:26:42,197
"(2021, 7)",9,08:35:22,233
"(2021, 8)",17,06:09:09,240
"(2021, 9)",22,08:48:45,240
"(2021, 10)",1,17:43:58,287


In [595]:
temp22 = pd.concat([sp3, total22])
temp22

,days,time,months
"(2022, 1)",6,19:03:00,392
"(2022, 2)",9,20:56:47,330
"(2022, 3)",3,12:08:33,361
"(2022, 4)",20,01:08:59,373
"(2022, 5)",25,11:31:03,400
"(2022, 6)",2,13:22:44,401
"(2022, 7)",15,05:53:35,453
"(2022, 8)",17,10:04:04,452
0,10,22:08:44.999999488,3165


In [596]:
temp01 = pd.concat([temp20, temp21])
temp02 = pd.concat([temp01, temp22])
temp02

,days,time,months
"(2020, 1)",14,06:04:49,161
"(2020, 2)",27,14:53:40,126
"(2020, 3)",20,18:42:13,140
"(2020, 4)",10,19:02:23,173
"(2020, 5)",8,16:05:39,223
"(2020, 6)",25,07:42:45,280
"(2020, 7)",18,17:47:42,331
"(2020, 8)",4,13:12:00,333
"(2020, 9)",27,09:02:08,262
"(2020, 10)",29,02:56:50,326


In [597]:
temp = temp02.copy()
temp

,days,time,months
"(2020, 1)",14,06:04:49,161
"(2020, 2)",27,14:53:40,126
"(2020, 3)",20,18:42:13,140
"(2020, 4)",10,19:02:23,173
"(2020, 5)",8,16:05:39,223
"(2020, 6)",25,07:42:45,280
"(2020, 7)",18,17:47:42,331
"(2020, 8)",4,13:12:00,333
"(2020, 9)",27,09:02:08,262
"(2020, 10)",29,02:56:50,326


In [598]:
temp.columns

Index(['days', 'time', 'months'], dtype='object')

In [599]:
temp = temp[['months', 'days', 'time']]
temp

,months,days,time
"(2020, 1)",161,14,06:04:49
"(2020, 2)",126,27,14:53:40
"(2020, 3)",140,20,18:42:13
"(2020, 4)",173,10,19:02:23
"(2020, 5)",223,8,16:05:39
"(2020, 6)",280,25,07:42:45
"(2020, 7)",331,18,17:47:42
"(2020, 8)",333,4,13:12:00
"(2020, 9)",262,27,09:02:08
"(2020, 10)",326,29,02:56:50


In [600]:
time = f'{hours}:{minutes}:{seconds}'
total = pd.DataFrame({'months':[months], 'days':[days], 'time':[time]})
total

,months,days,time
0,9100,23,15:6:51


In [601]:
temp = pd.concat([temp, total])
temp

,months,days,time
"(2020, 1)",161,14,06:04:49
"(2020, 2)",126,27,14:53:40
"(2020, 3)",140,20,18:42:13
"(2020, 4)",173,10,19:02:23
"(2020, 5)",223,8,16:05:39
"(2020, 6)",280,25,07:42:45
"(2020, 7)",331,18,17:47:42
"(2020, 8)",333,4,13:12:00
"(2020, 9)",262,27,09:02:08
"(2020, 10)",326,29,02:56:50


In [602]:
temp.to_excel('2020-22년_급속충전_년월별_총_충전시간.xlsx', encoding=True)

#### 년, 월별 총 충전시간

In [403]:
col = '충전시간'
ch_cal_time_by_ym = df2.groupby(['충전시작_년도', '충전시작_월'])[col].sum()
ch_cal_time_by_ym

충전시작_년도  충전시작_월
2020     1         4844 days 06:04:49
         2         3807 days 14:53:40
         3         4220 days 18:42:13
         4         5200 days 19:02:23
         5         6698 days 16:05:39
         6         8425 days 07:42:45
         7         9948 days 17:47:42
         8         9994 days 13:12:00
         9         7887 days 09:02:08
         10        9809 days 02:56:50
         11        9686 days 05:46:34
         12        8772 days 14:40:03
2021     1         6700 days 00:31:35
         2         5972 days 01:10:01
         3         6318 days 03:25:10
         4         5981 days 18:18:29
         5         5934 days 01:30:44
         6         5927 days 10:26:42
         7         6999 days 08:35:22
         8         7217 days 06:09:09
         9         7222 days 08:48:45
         10        8611 days 17:43:58
         11       10017 days 21:16:40
         12       11864 days 13:04:45
2022     1        11766 days 19:03:00
         2         9909 days 20:56

In [427]:
col = '충전시간'
ch_cal_time_by_y = df2.groupby(['충전시작_년도'])[col].sum()
ch_cal_time_by_y

충전시작_년도
2020             89296 days 01:56:46
2021             88766 days 15:01:20
2022   94960 days 22:08:44.999999488
Name: 충전시간, dtype: timedelta64[ns]

In [428]:
ch_cal_time_by_y = ch_cal_time_by_y.to_frame()
ch_cal_time_by_y

,충전시간
충전시작_년도,
2020,89296 days 01:56:46
2021,88766 days 15:01:20
2022,94960 days 22:08:44.999999488


In [434]:
ch_cal_time_by_y['충전시간'] = ch_cal_time_by_y['충전시간'].astype(str)
ch_cal_time_by_y

,충전시간
충전시작_년도,
2020,89296 days 01:56:46
2021,88766 days 15:01:20
2022,94960 days 22:08:44.999999488


In [435]:
ch_cal_time_by_y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 2020 to 2022
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   충전시간    3 non-null      object
dtypes: object(1)
memory usage: 48.0+ bytes


In [438]:
ch_cal_time_by_y = ch_cal_time_by_y['충전시간'].str.split(expand=True)
ch_cal_time_by_y

,0,1,2
충전시작_년도,,,
2020,89296,days,01:56:46
2021,88766,days,15:01:20
2022,94960,days,22:08:44.999999488


In [440]:
ch_cal_time_by_y = ch_cal_time_by_y.drop([1], axis=1)
ch_cal_time_by_y

,0,2
충전시작_년도,,
2020,89296,01:56:46
2021,88766,15:01:20
2022,94960,22:08:44.999999488


In [443]:
ch_cal_time_by_y[0] = ch_cal_time_by_y[0].astype(int)
ch_cal_time_by_y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 2020 to 2022
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       3 non-null      int32 
 1   2       3 non-null      object
dtypes: int32(1), object(1)
memory usage: 60.0+ bytes


In [444]:
temp2020 = ch_cal_time_by_y.iloc[0]
temp2021 = ch_cal_time_by_y.iloc[1]
temp2022 = ch_cal_time_by_y.iloc[2]

In [448]:
d = temp2022[0]
months = d // 30
days = d % 30
months, days

(3165, 10)

In [404]:
ch_cal_time_by_ym = ch_cal_time_by_ym.to_frame()
ch_cal_time_by_ym

충전시간
충전시작_년도 충전시작_월                    
2020    1       4844 days 06:04:49
        2       3807 days 14:53:40
        3       4220 days 18:42:13
        4       5200 days 19:02:23
        5       6698 days 16:05:39
        6       8425 days 07:42:45
        7       9948 days 17:47:42
        8       9994 days 13:12:00
        9       7887 days 09:02:08
        10      9809 days 02:56:50
        11      9686 days 05:46:34
        12      8772 days 14:40:03
2021    1       6700 days 00:31:35
        2       5972 days 01:10:01
        3       6318 days 03:25:10
        4       5981 days 18:18:29
        5       5934 days 01:30:44
        6       5927 days 10:26:42
        7       6999 days 08:35:22
        8       7217 days 06:09:09
        9       7222 days 08:48:45
        10      8611 days 17:43:58
        11     10017 days 21:16:40
        12     11864 days 13:04:45
2022    1      11766 days 19:03:00
        2       9909 days 20:56:47
        3      10833 days 12:08:33
        4      11210 days 01:08:59
        5      12025 days 11:31:03
        6      12032 days 13:22:44
        7      13605 days 05:53:35
        8      13577 days 10:04:04

In [405]:
temp1 = ch_cal_time_by_ym[col].dt.seconds.sum()
temp1

1264011

In [406]:
temp2 = ch_cal_time_by_ym[col].dt.days.sum()
temp2

273009

In [407]:
months = temp2 // 30
days = temp2 % 30
hours = temp1 // 3600
minutes = (temp1 % 3600) // 60
seconds = (temp1 % 3600) % 60
months, days, hours, minutes, seconds

(9100, 9, 351, 6, 51)

In [408]:
days += hours // 24
hours = hours % 24
days, hours

(23, 15)

In [409]:
ch_cal_time_by_ym[col] = ch_cal_time_by_ym[col].astype(str)
ch_cal_time_by_ym

충전시간
충전시작_년도 충전시작_월                     
2020    1        4844 days 06:04:49
        2        3807 days 14:53:40
        3        4220 days 18:42:13
        4        5200 days 19:02:23
        5        6698 days 16:05:39
        6        8425 days 07:42:45
        7        9948 days 17:47:42
        8        9994 days 13:12:00
        9        7887 days 09:02:08
        10       9809 days 02:56:50
        11       9686 days 05:46:34
        12       8772 days 14:40:03
2021    1        6700 days 00:31:35
        2        5972 days 01:10:01
        3        6318 days 03:25:10
        4        5981 days 18:18:29
        5        5934 days 01:30:44
        6        5927 days 10:26:42
        7        6999 days 08:35:22
        8        7217 days 06:09:09
        9        7222 days 08:48:45
        10       8611 days 17:43:58
        11      10017 days 21:16:40
        12      11864 days 13:04:45
2022    1       11766 days 19:03:00
        2        9909 days 20:56:47
        3       10833 days 12:08:33
        4       11210 days 01:08:59
        5       12025 days 11:31:03
        6       12032 days 13:22:44
        7       13605 days 05:53:35
        8       13577 days 10:04:04

In [410]:
temp = ch_cal_time_by_ym[col].str.split(expand=True)
temp

0     1         2
충전시작_년도 충전시작_월                       
2020    1        4844  days  06:04:49
        2        3807  days  14:53:40
        3        4220  days  18:42:13
        4        5200  days  19:02:23
        5        6698  days  16:05:39
        6        8425  days  07:42:45
        7        9948  days  17:47:42
        8        9994  days  13:12:00
        9        7887  days  09:02:08
        10       9809  days  02:56:50
        11       9686  days  05:46:34
        12       8772  days  14:40:03
2021    1        6700  days  00:31:35
        2        5972  days  01:10:01
        3        6318  days  03:25:10
        4        5981  days  18:18:29
        5        5934  days  01:30:44
        6        5927  days  10:26:42
        7        6999  days  08:35:22
        8        7217  days  06:09:09
        9        7222  days  08:48:45
        10       8611  days  17:43:58
        11      10017  days  21:16:40
        12      11864  days  13:04:45
2022    1       11766  days  19:03:00
        2        9909  days  20:56:47
        3       10833  days  12:08:33
        4       11210  days  01:08:59
        5       12025  days  11:31:03
        6       12032  days  13:22:44
        7       13605  days  05:53:35
        8       13577  days  10:04:04

In [411]:
temp = temp.drop([1], axis=1)
temp

0         2
충전시작_년도 충전시작_월                 
2020    1        4844  06:04:49
        2        3807  14:53:40
        3        4220  18:42:13
        4        5200  19:02:23
        5        6698  16:05:39
        6        8425  07:42:45
        7        9948  17:47:42
        8        9994  13:12:00
        9        7887  09:02:08
        10       9809  02:56:50
        11       9686  05:46:34
        12       8772  14:40:03
2021    1        6700  00:31:35
        2        5972  01:10:01
        3        6318  03:25:10
        4        5981  18:18:29
        5        5934  01:30:44
        6        5927  10:26:42
        7        6999  08:35:22
        8        7217  06:09:09
        9        7222  08:48:45
        10       8611  17:43:58
        11      10017  21:16:40
        12      11864  13:04:45
2022    1       11766  19:03:00
        2        9909  20:56:47
        3       10833  12:08:33
        4       11210  01:08:59
        5       12025  11:31:03
        6       12032  13:22:44
        7       13605  05:53:35
        8       13577  10:04:04

In [412]:
temp.columns = ['days', 'time']
temp

days      time
충전시작_년도 충전시작_월                 
2020    1        4844  06:04:49
        2        3807  14:53:40
        3        4220  18:42:13
        4        5200  19:02:23
        5        6698  16:05:39
        6        8425  07:42:45
        7        9948  17:47:42
        8        9994  13:12:00
        9        7887  09:02:08
        10       9809  02:56:50
        11       9686  05:46:34
        12       8772  14:40:03
2021    1        6700  00:31:35
        2        5972  01:10:01
        3        6318  03:25:10
        4        5981  18:18:29
        5        5934  01:30:44
        6        5927  10:26:42
        7        6999  08:35:22
        8        7217  06:09:09
        9        7222  08:48:45
        10       8611  17:43:58
        11      10017  21:16:40
        12      11864  13:04:45
2022    1       11766  19:03:00
        2        9909  20:56:47
        3       10833  12:08:33
        4       11210  01:08:59
        5       12025  11:31:03
        6       12032  13:22:44
        7       13605  05:53:35
        8       13577  10:04:04

In [413]:
temp['months'] = temp['days'].astype(int) // 30
temp['days'] = temp['days'].astype(int) % 30
temp

days      time  months
충전시작_년도 충전시작_월                        
2020    1         14  06:04:49     161
        2         27  14:53:40     126
        3         20  18:42:13     140
        4         10  19:02:23     173
        5          8  16:05:39     223
        6         25  07:42:45     280
        7         18  17:47:42     331
        8          4  13:12:00     333
        9         27  09:02:08     262
        10        29  02:56:50     326
        11        26  05:46:34     322
        12        12  14:40:03     292
2021    1         10  00:31:35     223
        2          2  01:10:01     199
        3         18  03:25:10     210
        4         11  18:18:29     199
        5         24  01:30:44     197
        6         17  10:26:42     197
        7          9  08:35:22     233
        8         17  06:09:09     240
        9         22  08:48:45     240
        10         1  17:43:58     287
        11        27  21:16:40     333
        12        14  13:04:45     395
2022    1          6  19:03:00     392
        2          9  20:56:47     330
        3          3  12:08:33     361
        4         20  01:08:59     373
        5         25  11:31:03     400
        6          2  13:22:44     401
        7         15  05:53:35     453
        8         17  10:04:04     452

In [414]:
temp.columns

Index(['days', 'time', 'months'], dtype='object')

In [415]:
temp = temp[['months', 'days', 'time']]
temp

months  days      time
충전시작_년도 충전시작_월                        
2020    1          161    14  06:04:49
        2          126    27  14:53:40
        3          140    20  18:42:13
        4          173    10  19:02:23
        5          223     8  16:05:39
        6          280    25  07:42:45
        7          331    18  17:47:42
        8          333     4  13:12:00
        9          262    27  09:02:08
        10         326    29  02:56:50
        11         322    26  05:46:34
        12         292    12  14:40:03
2021    1          223    10  00:31:35
        2          199     2  01:10:01
        3          210    18  03:25:10
        4          199    11  18:18:29
        5          197    24  01:30:44
        6          197    17  10:26:42
        7          233     9  08:35:22
        8          240    17  06:09:09
        9          240    22  08:48:45
        10         287     1  17:43:58
        11         333    27  21:16:40
        12         395    14  13:04:45
2022    1          392     6  19:03:00
        2          330     9  20:56:47
        3          361     3  12:08:33
        4          373    20  01:08:59
        5          400    25  11:31:03
        6          401     2  13:22:44
        7          453    15  05:53:35
        8          452    17  10:04:04

In [416]:
temp.columns

Index(['months', 'days', 'time'], dtype='object')

In [417]:
total = pd.DataFrame({'months':[months], 'days':[days], 'time':[f'{hours}:{minutes}:{seconds}']})
total

,months,days,time
0,9100,23,15:6:51


In [418]:
temp.to_excel('2020-22년_급속충전_년월별_총_충전시간.xlsx', encoding='utf-8')

#### 총 충전시간(계산)

In [603]:
col = '충전시간(계산)'
ch_cal_time_by_ym = df2.groupby(['충전시작_년도', '충전시작_월'])[col].sum()
ch_cal_time_by_ym.head()

충전시작_년도  충전시작_월
2020     1        5111 days 07:36:54
         2        4030 days 10:04:17
         3        4128 days 11:36:27
         4        4900 days 20:11:06
         5        6183 days 22:26:24
Name: 충전시간(계산), dtype: timedelta64[ns]

In [604]:
ch_cal_time_by_ym = ch_cal_time_by_ym.to_frame()
ch_cal_time_by_ym.head()

충전시간(계산)
충전시작_년도 충전시작_월                   
2020    1      5111 days 07:36:54
        2      4030 days 10:04:17
        3      4128 days 11:36:27
        4      4900 days 20:11:06
        5      6183 days 22:26:24

In [605]:
temp1 = ch_cal_time_by_ym[col].dt.seconds.sum()
temp1

1398469

In [606]:
temp2 = ch_cal_time_by_ym[col].dt.days.sum()
temp2

282067

In [607]:
months = temp2 // 30
days = temp2 % 30
hours = temp1 // 3600
minutes = (temp1 % 3600) // 60
seconds = (temp1 % 3600) % 60
months, days, hours, minutes, seconds

(9402, 7, 388, 27, 49)

In [608]:
days += hours // 24
hours = hours % 24
days, hours

(23, 4)

In [609]:
ch_cal_time_by_ym[col] = ch_cal_time_by_ym[col].astype(str)
temp = ch_cal_time_by_ym[col].str.split(expand=True)

In [610]:
temp = temp.drop([1], axis=1)
temp.columns = ['days', 'time']
temp['months'] = temp['days'].astype(int) // 30
temp['days'] = temp['days'].astype(int) % 30

In [611]:
temp.columns

Index(['days', 'time', 'months'], dtype='object')

In [612]:
sp1 = temp.iloc[:12, :]
sp1

days      time  months
충전시작_년도 충전시작_월                        
2020    1         11  07:36:54     170
        2         10  10:04:17     134
        3         18  11:36:27     137
        4         10  20:11:06     163
        5          3  22:26:24     206
        6         11  18:02:20     251
        7          3  12:15:12     292
        8         29  04:45:22     296
        9          6  07:09:15     237
        10        12  04:42:48     301
        11        23  08:37:36     316
        12        28  17:01:32     293

In [613]:
sp2 = temp.iloc[12:24, :]
sp2

days      time  months
충전시작_년도 충전시작_월                        
2021    1         10  17:06:06     232
        2          8  23:23:36     209
        3         27  16:40:25     219
        4         27  11:25:41     208
        5         19  08:51:42     207
        6         23  16:48:38     207
        7          0  13:25:41     245
        8         21  11:25:24     264
        9         16  00:14:44     268
        10        22  22:42:28     319
        11        14  09:48:00     366
        12         3  17:25:22     430

In [614]:
sp3 = temp.iloc[24:, :]
sp3

days      time  months
충전시작_년도 충전시작_월                        
2022    1         16  06:53:00     425
        2         20  18:15:50     358
        3          0  02:18:10     394
        4          7  02:13:02     406
        5         16  15:54:21     437
        6         26  21:10:27     435
        7         12  02:56:44     483
        8          6  04:59:15     479

In [615]:
ch_cal_time_by_y = df2.groupby(['충전시작_년도'])[col].sum()
ch_cal_time_by_y.head()

충전시작_년도
2020    84050 days 00:29:12.999999488
2021    95417 days 01:17:47.000000512
2022   102616 days 02:40:48.999999488
Name: 충전시간(계산), dtype: timedelta64[ns]

In [616]:
ch_cal_time_by_y = ch_cal_time_by_y.to_frame()
ch_cal_time_by_y[col] = ch_cal_time_by_y[col].astype(str)
ch_cal_time_by_y

,충전시간(계산)
충전시작_년도,
2020,84050 days 00:29:12.999999488
2021,95417 days 01:17:47.000000512
2022,102616 days 02:40:48.999999488


In [617]:
ch_cal_time_by_y = ch_cal_time_by_y[col].str.split(expand=True)
ch_cal_time_by_y

,0,1,2
충전시작_년도,,,
2020,84050,days,00:29:12.999999488
2021,95417,days,01:17:47.000000512
2022,102616,days,02:40:48.999999488


In [618]:
ch_cal_time_by_y = ch_cal_time_by_y.drop([1], axis=1)
ch_cal_time_by_y

,0,2
충전시작_년도,,
2020,84050,00:29:12.999999488
2021,95417,01:17:47.000000512
2022,102616,02:40:48.999999488


In [619]:
ch_cal_time_by_y[0] = ch_cal_time_by_y[0].astype(int)
ch_cal_time_by_y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 2020 to 2022
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       3 non-null      int32 
 1   2       3 non-null      object
dtypes: int32(1), object(1)
memory usage: 60.0+ bytes


In [620]:
temp20 = ch_cal_time_by_y.iloc[0]
temp21 = ch_cal_time_by_y.iloc[1]
temp22 = ch_cal_time_by_y.iloc[2]

In [621]:
d20, t20 = temp20[0], temp20[2]
months20 = d20 // 30
days20 = d20 % 30
months20, days20, t20

(2801, 20, '00:29:12.999999488')

In [622]:
d21, t21 = temp21[0], temp21[2]
months21 = d21 // 30
days21 = d21 % 30
months21, days21, t21

(3180, 17, '01:17:47.000000512')

In [623]:
d22,t22 = temp22[0], temp22[2]
months22 = d22 // 30
days22 = d22 % 30
months22, days22, t22

(3420, 16, '02:40:48.999999488')

In [624]:
sp1.columns

Index(['days', 'time', 'months'], dtype='object')

In [625]:
total20 = pd.DataFrame({'days':[days20], 'time':[t20], 'months':[months20]})
total20

,days,time,months
0,20,00:29:12.999999488,2801


In [626]:
total21 = pd.DataFrame({'days':[days21], 'time':[t21], 'months':[months21]})
total21

,days,time,months
0,17,01:17:47.000000512,3180


In [627]:
total22 = pd.DataFrame({'days':[days22], 'time':[t22], 'months':[months22]})
total22

,days,time,months
0,16,02:40:48.999999488,3420


In [628]:
temp20 = pd.concat([sp1, total20])
temp20

,days,time,months
"(2020, 1)",11,07:36:54,170
"(2020, 2)",10,10:04:17,134
"(2020, 3)",18,11:36:27,137
"(2020, 4)",10,20:11:06,163
"(2020, 5)",3,22:26:24,206
"(2020, 6)",11,18:02:20,251
"(2020, 7)",3,12:15:12,292
"(2020, 8)",29,04:45:22,296
"(2020, 9)",6,07:09:15,237
"(2020, 10)",12,04:42:48,301


In [629]:
temp21 = pd.concat([sp2, total21])
temp21

,days,time,months
"(2021, 1)",10,17:06:06,232
"(2021, 2)",8,23:23:36,209
"(2021, 3)",27,16:40:25,219
"(2021, 4)",27,11:25:41,208
"(2021, 5)",19,08:51:42,207
"(2021, 6)",23,16:48:38,207
"(2021, 7)",0,13:25:41,245
"(2021, 8)",21,11:25:24,264
"(2021, 9)",16,00:14:44,268
"(2021, 10)",22,22:42:28,319


In [630]:
temp22 = pd.concat([sp3, total22])
temp22

,days,time,months
"(2022, 1)",16,06:53:00,425
"(2022, 2)",20,18:15:50,358
"(2022, 3)",0,02:18:10,394
"(2022, 4)",7,02:13:02,406
"(2022, 5)",16,15:54:21,437
"(2022, 6)",26,21:10:27,435
"(2022, 7)",12,02:56:44,483
"(2022, 8)",6,04:59:15,479
0,16,02:40:48.999999488,3420


In [631]:
temp01 = pd.concat([temp20, temp21])
temp02 = pd.concat([temp01, temp22])
temp02

,days,time,months
"(2020, 1)",11,07:36:54,170
"(2020, 2)",10,10:04:17,134
"(2020, 3)",18,11:36:27,137
"(2020, 4)",10,20:11:06,163
"(2020, 5)",3,22:26:24,206
"(2020, 6)",11,18:02:20,251
"(2020, 7)",3,12:15:12,292
"(2020, 8)",29,04:45:22,296
"(2020, 9)",6,07:09:15,237
"(2020, 10)",12,04:42:48,301


In [632]:
temp = temp02.copy()
temp

,days,time,months
"(2020, 1)",11,07:36:54,170
"(2020, 2)",10,10:04:17,134
"(2020, 3)",18,11:36:27,137
"(2020, 4)",10,20:11:06,163
"(2020, 5)",3,22:26:24,206
"(2020, 6)",11,18:02:20,251
"(2020, 7)",3,12:15:12,292
"(2020, 8)",29,04:45:22,296
"(2020, 9)",6,07:09:15,237
"(2020, 10)",12,04:42:48,301


In [633]:
temp.columns

Index(['days', 'time', 'months'], dtype='object')

In [634]:
temp = temp[['months', 'days', 'time']]

In [635]:
time = f'{hours}:{minutes}:{seconds}'
total = pd.DataFrame({'months':[months], 'days':[days], 'time':[time]})
total

,months,days,time
0,9402,23,4:27:49


In [636]:
temp = pd.concat([temp, total])
temp

,months,days,time
"(2020, 1)",170,11,07:36:54
"(2020, 2)",134,10,10:04:17
"(2020, 3)",137,18,11:36:27
"(2020, 4)",163,10,20:11:06
"(2020, 5)",206,3,22:26:24
"(2020, 6)",251,11,18:02:20
"(2020, 7)",292,3,12:15:12
"(2020, 8)",296,29,04:45:22
"(2020, 9)",237,6,07:09:15
"(2020, 10)",301,12,04:42:48


In [637]:
temp.to_excel('2020-22년_급속충전_년월별_총_충전시간(계산).xlsx', encoding=True)

## 주소 분리
- 신주소, 구주소

In [94]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 27 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int16          
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_

In [772]:
df2['주소'].head()

0    강원도 인제군 상남면 하남리 339-7(양양방향)
1    강원도 인제군 상남면 하남리 339-7(양양방향)
2    강원도 인제군 상남면 하남리 339-7(양양방향)
3    강원도 인제군 상남면 하남리 339-7(양양방향)
4        광주광역시 북구 첨단 벤처로 108번길 9
Name: 주소, dtype: object

In [773]:
df2['주소'][:10]

0    강원도 인제군 상남면 하남리 339-7(양양방향)
1    강원도 인제군 상남면 하남리 339-7(양양방향)
2    강원도 인제군 상남면 하남리 339-7(양양방향)
3    강원도 인제군 상남면 하남리 339-7(양양방향)
4        광주광역시 북구 첨단 벤처로 108번길 9
5        전라남도 순천시 해룡면 남가리 733-73
6        전라남도 순천시 해룡면 남가리 733-73
7        전라남도 순천시 해룡면 남가리 733-73
8          전라남도 구례군 산동면 산업로 2319
9          전라남도 구례군 산동면 산업로 2319
Name: 주소, dtype: object

In [96]:
ad = df2['주소'].copy()
ad.info()

<class 'pandas.core.series.Series'>
Int64Index: 11613449 entries, 0 to 11665821
Series name: 주소
Non-Null Count     Dtype 
--------------     ----- 
11613449 non-null  object
dtypes: object(1)
memory usage: 177.2+ MB


In [99]:
temp = ad.str.split(expand=True)
temp

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,강원도,인제군,상남면,하남리,339-7(양양방향),None,None,None,None,None,None,None,None
1,강원도,인제군,상남면,하남리,339-7(양양방향),None,None,None,None,None,None,None,None
2,강원도,인제군,상남면,하남리,339-7(양양방향),None,None,None,None,None,None,None,None
3,강원도,인제군,상남면,하남리,339-7(양양방향),None,None,None,None,None,None,None,None
4,광주광역시,북구,첨단,벤처로,108번길,9,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11665817,경기도,용인시,수지구,경부고속도로,400,(죽전동),None,None,None,None,None,None,None
11665818,강원도,원주시,단구로,150,None,None,None,None,None,None,None,None,None
11665819,인천광역시,서구,당하동,1097-5,검단4동,주민센터,옆,None,None,None,None,None,None
11665820,대구광역시,남구,봉덕동,1272-18,None,None,None,None,None,None,None,None,None


In [100]:
temp[0].unique()

array(['강원도', '광주광역시', '전라남도', '전라북도', '제주특별자치도', '세종특별자치시', '서울특별시',
       '대전광역시', '경기도', '인천광역시', '경상남도', '충청북도', '경상북도', '충청남도', '울산광역시',
       '대구광역시', '부산광역시'], dtype=object)

In [101]:
temp[1].unique()

array(['인제군', '북구', '순천시', '구례군', '장성군', '부안군', '영암군', '무안군', '남원시',
       '정읍시', '무주군', '전주시', '제주시', '서귀포시', '연기면', '영등포구', '서구', '동구',
       '대덕구', '중구', '유성구', '양주시', '파주시', '고양시', '부천시', '수원시', '안산시',
       '하남시', '마포구', '성동구', '남동구', '거제시', '원주시', '강릉시', '동해시', '삼척시',
       '횡성군', '평창군', '철원군', '청주시', '충주시', '제천시', '보은군', '영동군', '증평군',
       '괴산군', '단양군', '이천시', '여주시', '용인시', '구리시', '화순군', '완주군', '김해시',
       '성주군', '시흥시', '양산시', '영월군', '정선군', '춘천시', '화천군', '가평군', '화성시',
       '금산군', '함양군', '문경시', '창녕군', '함안군', '창원시', '진주시', '통영시', '남구',
       '수성구', '달서구', '울진군', '칠곡군', '포항시', '김천시', '영주시', '달성군', '논산시',
       '보령시', '서천군', '천안시', '홍성군', '곡성군', '음성군', '경주시', '광산구', '여수시',
       '강서구', '도봉구', '관악구', '기장군', '고흥군', '광양시', '보성군', '완도군', '나주시',
       '고창군', '군산시', '임실군', '거창군', '고성군', '밀양시', '사천시', '합천군', '경산시',
       '공주시', '서산시', '계룡시', '당진시', '태안군', '익산시', '진안군', '강진군', '성남시',
       '부여군', '구미시', '의성군', '군위군', '광주시', '양평군', '동두천시', '안성시', '군포시',
       '김포시', '남양주시', 

In [107]:
exc_idx = []
exc_s = []
p = re.compile('\w+로$')
for i, s in enumerate(temp[1].to_list()):
    if p.match(s):
        exc_idx.append(i)
        exc_s.append(s)
len(exc_idx), len(exc_s)

(17582, 17582)

In [109]:
set(exc_s)

{'가름로',
 '갈매로',
 '국세청로',
 '나성남로',
 '다솜2로',
 '다솜3로',
 '다솜로',
 '도움5로',
 '도움6로',
 '보듬6로',
 '정부2청사로',
 '한누리대로'}

In [110]:
temp[2].unique()

array(['상남면', '첨단', '해룡면', ..., '장유면', '첨단과기로176번길', '정서진1로'],
      dtype=object)

In [111]:
temp = ad.to_list()
len(temp)

11613449

In [126]:
temp[:10]

['강원도 인제군 상남면 하남리 339-7(양양방향)',
 '강원도 인제군 상남면 하남리 339-7(양양방향)',
 '강원도 인제군 상남면 하남리 339-7(양양방향)',
 '강원도 인제군 상남면 하남리 339-7(양양방향)',
 '광주광역시 북구 첨단 벤처로 108번길 9',
 '전라남도 순천시 해룡면 남가리 733-73',
 '전라남도 순천시 해룡면 남가리 733-73',
 '전라남도 순천시 해룡면 남가리 733-73',
 '전라남도 구례군 산동면 산업로 2319',
 '전라남도 구례군 산동면 산업로 2319']

In [202]:
# 오류 주소
    # 충청북도 단양군 적성면 기동리

old_adr = []
old_idx = []
new_adr = []
new_idx = []
# new_p = re.compile('[가-힣]+로{1}|[가-힣]+로[0-9]+|[가-힣]+길|[가-힣0-9]+번길')
old_p = re.compile('[가-힣]+동\s|[가-힣]+동[0-9]+|[가-힣]+리\s|\s[가-힣]+리')
for i, s in enumerate(temp):
    # if new_p.search(s):
    if old_p.search(s):
        old_adr.append(s)
        old_idx.append(i)
    else:
        new_adr.append(s)
        new_idx.append(i)

print(len(new_idx))
print(len(old_idx))
print(len(temp))
print(len(new_idx) + len(old_idx) == len(temp))

7709259
3904190
11613449
True


In [203]:
count = 0
for i in new_adr:
    if '기동리' in i:
        count += 1
print(count)

0


In [204]:
for i in new_adr:
    if '구로구' in i:
        print(i)
        break

서울특별시 구로구 경인로 661  지하2층 L


In [205]:
for i in new_adr:
    if '구로구 개봉동' in i:
        print(i)
        break

In [212]:
start_idx = 5000000
new_adr[start_idx:start_idx + 10]

['경상남도 고성군 대가면 통영대전고속도로 29',
 '충청남도 금산군 군북면 어필각로 300',
 '경상남도 고성군 대가면 통영대전고속도로 32',
 '인천광역시 동구 염전로 30',
 '강원도 원주시 신림면 치악로 416',
 '경상남도 김해시 진영읍 김해대로 440',
 '대구광역시 동구 신천3동 146-15',
 '전라북도 무주군 안성면 통영대전고속도로 148',
 '경상북도 상주시 화서면 터골길 51-25',
 '전라남도 여수시 돌산읍 강남로 31']

In [213]:
start_idx = 1500000
old_adr[start_idx:start_idx + 10]

['경상북도 경주시 현곡면 오류리 277-9',
 '경상남도 밀양시 산내면 삼양리 190-1번지',
 '충청남도 아산시 온천동 1880 충청남도 아산시 온천동 1880',
 '경상북도 의성군 단밀면 낙정리',
 '제주특별자치도 제주시 구좌읍 세화리 1500-41',
 '경상북도 영덕군 영덕읍 우곡리 523',
 '전라북도 정읍시 상동 258-4',
 '경기도 안산시 단원구 선부동 703-2',
 '충청남도 서산시 해미면 읍내리 353',
 '충청북도 음성군 감곡면 오궁리 533-32 정면 우측 주차장']

In [214]:
new_adr_df = df2.iloc[new_idx]
old_adr_df = df2.iloc[old_idx]

In [215]:
new_adr_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7709259 entries, 4 to 11665821
Data columns (total 27 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int16          
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_초

In [216]:
old_adr_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3904190 entries, 0 to 11665820
Data columns (total 27 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int16          
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_초

##### 신주소, 구주소 분리 파일 출력

In [218]:
new_adr_df.to_csv('2020-22년_급속충전_신주소.csv', encoding='utf-8')
old_adr_df.to_csv('2020-22년_급속충전_구주소.csv', encoding='utf-8')

##### 신주소, 구주소 로드(다시 시작)

In [ ]:
# new_adr_df = pd.read_csv('2020-22년_급속충전_신주소.csv', index_col=0)
# old_adr_df = pd.read_csv('2020-22년_급속충전_구주소.csv', index_col=0)

### 신주소 상세 분리

In [219]:
temp = new_adr_df['주소'].str.split(expand=True)
temp.head()

MemoryError: Unable to allocate 765. MiB for an array with shape (7709259, 13) and data type object